# 0 Project Barley

Along this project we will work on various Barley's cultivar starting from Hv_Morex. The aim of this project is to further understand the importance of introns within genes and within Untranslated Regions (UTRs) and their differences. 

Another not secondary objective is to build a pipeline as much reproducible as possible. This notebook is meant to give readers a crystal clear view of how we proceeded to implement our work. 

All the work has been performed on a Linux machine running _Ubuntu 20.04.2 LTS x86_64_.

# 1 Working environment

**Before running the notebook** be sure to activate the conda environment shared with you:
- $ conda activate Pedroni_Thesis.yml

In [ ]:
### --- This command can be installed with: $ sudo apt-get install neofetch
### --- It is used to show the software/hardware characteristics of the machine used to complete this project.
! neofetch | grep "OS\|Shell\|DE\|CPU\|GPU\|Memory"

In [ ]:
### --- This command can be installed using 
### --- This command is used to show you how directories should be organized to better be able to follow this work.
### --- The root directory of this project is called 'Project Barley'
! tree -d

# 2 Implementing an intron analysis on data from a long-read sequence assembly in Barley

The study I am referring to can be found [here](https://academic.oup.com/plcell/advance-article/doi/10.1093/plcell/koab077/6169005) while all the data can be found [here](https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/21172880-2956-4cbb-ab2c-5c00bceb08a2/0). 

## 2.1 Hv_Morex HC

### 2.1.1 Collecting Data

In [863]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Hv_Morex.pgsb.Jul2020.HC.gff3 https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/2492a5a9-08a8-4022-b6ad-9b056a00f64f/1/DOWNLOAD

--2021-09-04 12:39:51--  https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/2492a5a9-08a8-4022-b6ad-9b056a00f64f/1/DOWNLOAD
Resolving doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)... 194.94.136.144
Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59307477 (57M) [text/plain]
Saving to: ‘Data/raw/Hv_Morex.pgsb.Jul2020.HC.gff3’

w/Hv_Morex.pgsb.Jul  25%[====>               ]  14,46M   828KB/s    eta 48s    ^C


### 2.1.2 Processing data

In [864]:
### --- Importing the libraries needed to handle data and visualize them
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

In [865]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Hv_Morex.pgsb.Jul2020.HC.gff3  > Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.gff3

In [866]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.csv

In [867]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Hv_MorexHC = pd.read_csv('Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
0,chr1H,pgsb,gene,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030
1,chr1H,pgsb,mRNA,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030.1
2,chr1H,pgsb,exon,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030.1.exon1
3,chr1H,pgsb,CDS,76744,77373.0,.,+,0,ID=HORVU.MOREX.r3.1HG0000030.1.CDS1
4,chr1H,pgsb,gene,78284,81892.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040


In [868]:
### --- Building a separate dataframe containing all exons
exon_Hv_MorexHC = df_Hv_MorexHC.loc[df_Hv_MorexHC['type'].isin(['exon'])]
exon_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
2,chr1H,pgsb,exon,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030.1.exon1
7,chr1H,pgsb,exon,78284,78954.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.exon1
9,chr1H,pgsb,exon,79063,79104.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.exon2
11,chr1H,pgsb,exon,79609,79676.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.exon3
13,chr1H,pgsb,exon,79757,79799.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.exon4


In [869]:
### --- Exporting the exon dataframe to a tsv file 
exon_Hv_MorexHC.to_csv('Data/Hv_MorexHC_exon.tsv',sep='\t',index=False,header=False)

In [870]:
### --- Keeping in a separated dataframe the forward strands
forw_Hv_MorexHC = df_Hv_MorexHC.loc[df_Hv_MorexHC['strand'].isin(['+'])]
forw_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
0,chr1H,pgsb,gene,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030
1,chr1H,pgsb,mRNA,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030.1
2,chr1H,pgsb,exon,76744,77373.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000030.1.exon1
3,chr1H,pgsb,CDS,76744,77373.0,.,+,0,ID=HORVU.MOREX.r3.1HG0000030.1.CDS1
48,chr1H,pgsb,gene,132221,138736.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000060


In [871]:
### --- Keeping in a separated dataframe the reverse strands
rev_Hv_MorexHC = df_Hv_MorexHC.loc[df_Hv_MorexHC['strand'].isin(['-'])]
rev_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
4,chr1H,pgsb,gene,78284,81892.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040
5,chr1H,pgsb,mRNA,78284,81892.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1
6,chr1H,pgsb,three_prime_UTR,78284,78510.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.three_prime_UTR1
7,chr1H,pgsb,exon,78284,78954.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.exon1
8,chr1H,pgsb,CDS,78511,78954.0,.,-,0,ID=HORVU.MOREX.r3.1HG0000040.1.CDS1


In [872]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Hv_MorexHC = forw_Hv_MorexHC.loc[df_Hv_MorexHC['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
50,chr1H,pgsb,five_prime_UTR,132221,132375.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000060.1.five_prime_UTR1
93,chr1H,pgsb,three_prime_UTR,138505,138736.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000060.1.three_prime_UTR2
97,chr1H,pgsb,five_prime_UTR,146607,146749.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000070.1.five_prime_UTR1
98,chr1H,pgsb,five_prime_UTR,146981,146996.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000070.1.five_prime_UTR2
103,chr1H,pgsb,three_prime_UTR,148229,148562.0,.,+,.,ID=HORVU.MOREX.r3.1HG0000070.1.three_prime_UTR3


In [873]:
### --- Writing to a tsv formatted file the UTR_df_forw dataframe
UTR_forw_Hv_MorexHC.to_csv('Data/Hv_MorexHC_UTRforw.tsv',sep='\t',index=False,header=False)

In [874]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Hv_MorexHC = rev_Hv_MorexHC.loc[df_Hv_MorexHC['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Hv_MorexHC.head()

,chr,source,type,start,end,score,strand,phase,attributes
6,chr1H,pgsb,three_prime_UTR,78284,78510.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.three_prime_UTR1
27,chr1H,pgsb,five_prime_UTR,81706,81892.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000040.1.five_prime_UTR2
30,chr1H,pgsb,three_prime_UTR,84091,84317.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000050.1.three_prime_UTR1
47,chr1H,pgsb,five_prime_UTR,86846,87063.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000050.1.five_prime_UTR2
126,chr1H,pgsb,three_prime_UTR,156767,157203.0,.,-,.,ID=HORVU.MOREX.r3.1HG0000090.1.three_prime_UTR1


In [875]:
### --- Writing to a tsv formatted file the UTR_df_rev dataframe
UTR_rev_Hv_MorexHC.to_csv('Data/Hv_MorexHC_UTRrev.tsv',sep='\t',index=False,header=False)

### 2.1.3 Extracting introns from the whole genome

In [876]:
### --- Function to extract introns from the whole genome
### --- Infile is a file like Hv_MorexHC_exon.tsv 
### --- Outfile will be written thanks to this function
### --- If there are two or more exons belonging to the same mRNA one after the other it compute the introns separating them
def extract_tot_introns(infile, outfile):
    introns = open(outfile, 'a') # Output/Hv_Morex_introns.tsv
    introns.write('ID'+'\t'+'type'+'\t'+'start'+'\t'+'end'+'\t'+'length') 
    with open(infile) as f: # Data/Hv_Morex_exon.tsv
        lines = f.readlines()
        for i in range(0, len(lines)):
            if i+1 == len(lines): # This is to avoid out of range error
                break 
            else:
                line = lines[i]
                line = line.rstrip()
                line = line.split()
                next_line = lines[i+1]
                next_line = lines[i+1].rstrip()
                next_line = lines[i+1].split()
                if '.'.join(line[8].split('.')[0:5]) != '.'.join(next_line[8].split('.')[0:5]): continue # Checking if the next exon has the same ID of the one we are at
                #print('The intron coordinates at %s are from %d to %d and the intron length is %d.' % (line[8][:30], int(line[4])+1, int(next_line[3])-1, int(next_line[3])-1 - int(line[4])+1))
                introns.write('\n'+'.'.join(line[8].split('.')[0:5])+'\t'+ 'intron ' +'\t'+str(int(line[4])+1)+'\t'+str(int(next_line[3])-1)+'\t'+str(int(next_line[3])-1-int(line[4])+1))
    introns.close()


In [877]:
### --- Extracting introns from Hv_MorexHC
extract_tot_introns('Data/Hv_MorexHC_exon.tsv', 'Output/Hv_MorexHC_introns.tsv')

ValueError: invalid literal for int() with base 10: '78954.0'

In [ ]:
introns_Hv_MorexHC = pd.read_csv('Output/Hv_MorexHC_introns.tsv', sep = '\t')
introns_Hv_MorexHC[:100]

In [ ]:
### --- Function to count introns within a dataframe made of exons formatted like exon_Hv_MorexHC
### --- If I use i[:28] I consider the gene, If I use i[:31] I'm at level of mRNA
### --- Actually using 31 I would avoid the problem of certain genes having >= 10 transcripts
def counting_introns_type(in_dataframe):
    counts = dict()
    for i in in_dataframe['attributes']:
        counts['.'.join(i.split('.')[0:5])] = counts.get('.'.join(i.split('.')[0:5]), 0) + 1
    return counts

In [ ]:
### --- Counting introns type in Hv_MorexHC
introns_dictionary_Hv_MorexHC = counting_introns_type(exon_Hv_MorexHC)

In [ ]:
### --- Function to check intronless (no introns), intronpoor (<= 3 introns) and intron rich (> 3 introns)
### --- This division is made on the following assumption: n° introns = n° exons - 1
def splitting_introns_type(dictionary):
    intronless = 0
    intronpoor = 0
    intronrich = 0
    intronless_list = []
    intronpoor_list = []
    intronrich_list = []
    for i in dictionary:
        if dictionary[i] == 1:
            intronless += 1
            intronless_list.append(i)
        elif dictionary[i]>1 and dictionary[i]<= 4:
            intronpoor += 1
            intronpoor_list.append(i)
        elif dictionary[i] > 4:
            intronrich += 1
            intronrich_list.append(i)

    print('Intronless are: %d' % intronless)
    print('Intronpoor are: %d' % intronpoor)
    print('Intronrich are: %d' % intronrich)
    
    return (intronless, intronpoor, intronrich, intronless_list, intronpoor_list, intronrich_list)

In [ ]:
n_intronless_Hv_MorexHC, n_intronpoor_Hv_MorexHC, n_intronrich_Hv_MorexHC, intronless_Hv_MorexHC, intronpoor_Hv_MorexHC, intronrich_Hv_MorexHC = splitting_introns_type(introns_dictionary_Hv_MorexHC)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data = [11537, 12997, 11293]
fig = plt.figure(figsize =(10, 7))
plt.pie(data, labels = genes, autopct='%1.0f%%')
plt.title("Genes Hv_MorexHC Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Hv_MorexHC gene IDs to check the number of transcripts using command line
intronless_file = open('Output/Hv_MorexHC_intronlessIDs.txt', 'a')
intronpoor_file = open('Output/Hv_MorexHC_intronpoorIDs.txt', 'a')
intronrich_file = open('Output/Hv_MorexHC_intronrichIDs.txt', 'a')
for i in range(len(intronless_Hv_MorexHC)):
    if i == len(intronless_Hv_MorexHC) -1:
        intronless_file.write(intronless_Hv_MorexHC[i])
    else:
        intronless_file.write(intronless_Hv_MorexHC[i] + '\n')
intronless_file.close()

for i in range(len(intronpoor_Hv_MorexHC)):
    if i == len(intronpoor_Hv_MorexHC) -1:
        intronpoor_file.write(intronpoor_Hv_MorexHC[i])
    else:
        intronpoor_file.write(intronpoor_Hv_MorexHC[i] + '\n')
intronpoor_file.close()

for i in range(len(intronrich_Hv_MorexHC)):
    if i == len(intronrich_Hv_MorexHC) -1:
        intronrich_file.write(intronrich_Hv_MorexHC[i])
    else:
        intronrich_file.write(intronrich_Hv_MorexHC[i]+'\n')
intronrich_file.close()


In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Hv_MorexHC = ! grep -f Output/Hv_MorexHC_intronlessIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Hv_MorexHC = ! grep -f Output/Hv_MorexHC_intronpoorIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Hv_MorexHC = ! grep -f Output/Hv_MorexHC_intronrichIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.HC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Hv_MorexHC = int(n_mRNA_intronless_Hv_MorexHC[0])/n_intronless_Hv_MorexHC
print(avg_mRNA_intronless_Hv_MorexHC)
avg_mRNA_intronpoor_Hv_MorexHC = int(n_mRNA_intronpoor_Hv_MorexHC[0])/n_intronpoor_Hv_MorexHC
print(avg_mRNA_intronpoor_Hv_MorexHC)
avg_mRNA_intronrich_Hv_MorexHC = int(n_mRNA_intronrich_Hv_MorexHC[0])/n_intronrich_Hv_MorexHC
print(avg_mRNA_intronrich_Hv_MorexHC)

### 2.1.4 Extracting introns from UTR regions

In [ ]:
### --- Function to extract introns from UTR regions.
### --- Infile is a file like Hv_Morex_UTRforw_introns.tsv 
### --- Outfile will be written thanks to this function
### --- Being all on the same strand when two or more UTRs of the same type are one row after the other it means there is a intron separating them
def extract_UTR_introns(infile, outfile):
    UTR_introns_forw = open(outfile, 'a')
    UTR_introns_forw.write('ID'+'\t'+'type'+'\t'+'start'+'\t'+'end'+'\t'+'length') # Defining the header
    with open(infile) as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            if i+1 == len(lines):
                break
            else:
                line = lines[i]
                line = line.rstrip()
                line = line.split()
                next_line = lines[i+1]
                next_line = lines[i+1].rstrip()
                next_line = lines[i+1].split()
                if line[2] != next_line[2] or '.'.join(line[8].split('.')[0:5]) != '.'.join(next_line[8].split('.')[0:5]): continue
                #print('The intron coordinates at %s are from %d to %d and the intron length is %d.' % ('.'.join(line[8].split('.')[0:5]), int(line[4])+1, int(next_line[3])-1, int(next_line[3])-1 - int(line[4])+1))
                UTR_introns_forw.write('\n'+'.'.join(line[8].split('.')[0:5])+'\t'+ 'intron '+line[2]+'\t'+str(int(line[4])+1)+'\t'+str(int(next_line[3])-1)+'\t'+str(int(next_line[3])-1-int(line[4])+1))
    UTR_introns_forw.close()

In [ ]:
### --- Extracting introns from UTR forward Hv_MorexHC
extract_UTR_introns('Data/Hv_MorexHC_UTRforw.tsv', 'Output/Hv_MorexHC_UTRforw_introns.tsv')

In [ ]:
### --- Extracting introns from UTR reverse Hv_MorexHC
extract_UTR_introns('Data/Hv_MorexHC_UTRrev.tsv', 'Output/Hv_MorexHC_UTRrev_introns.tsv')

In [ ]:
introns_UTR_forw_Hv_MorexHC = pd.read_csv('Output/Hv_MorexHC_UTRforw_introns.tsv', sep='\t')
introns_UTR_forw_Hv_MorexHC.head()

In [ ]:
introns_UTR_rev_Hv_MorexHC = pd.read_csv('Output/Hv_MorexHC_UTRrev_introns.tsv', sep = '\t')
introns_UTR_rev_Hv_MorexHC.head()

In [ ]:
### --- Concatenating the UTR_introns to build a unique dataframe
frames = [introns_UTR_forw_Hv_MorexHC, introns_UTR_rev_Hv_MorexHC]
introns_UTR_Hv_MorexHC = pd.concat(frames)

In [ ]:
### --- Setting figure and font size
plt.rcParams["figure.figsize"] = (20, 20)
plt.rcParams["font.size"] = 15

In [ ]:
### --- Performing some basic statistics 
introns_UTR_Hv_MorexHC['length'].describe()

In [ ]:
### --- Plotting the distribution of UTR introns length
introns_UTR_Hv_MorexHC['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of UTR introns length without outliers
introns_UTR_Hv_MorexHC.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

In [ ]:
### --- splitting in introns 3'UTR and 5'UTR to eventually if there are differences among the two
introns_UTR3_Hv_MorexHC = introns_UTR_Hv_MorexHC.loc[introns_UTR_Hv_MorexHC['type'].isin(['intron three_prime_UTR'])]
introns_UTR5_Hv_MorexHC = introns_UTR_Hv_MorexHC.loc[introns_UTR_Hv_MorexHC['type'].isin(['intron five_prime_UTR'])]


In [ ]:
introns_UTR3_Hv_MorexHC

In [ ]:
introns_UTR3_Hv_MorexHC['length'].describe()

In [ ]:
introns_UTR5_Hv_MorexHC

In [ ]:
introns_UTR5_Hv_MorexHC['length'].describe()

In [ ]:
### --- Plotting the distribution of introns_UTR vs total UTRs
labels_Hv_MorexHC = ['UTRs', 'Introns_5UTR', 'Introns_3UTR']
UTR_data_Hv_MorexHC = [ 20666+19938-(3417+1632), 3417, 1632]
fig = plt.figure(figsize =(10, 7))
plt.pie(UTR_data_Hv_MorexHC, labels = labels_Hv_MorexHC, autopct='%1.0f%%')
plt.title("UTRs Hv_MorexHC Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

### 2.1.5 Removing UTR introns from the whole introns dataframe

In [ ]:
### --- Checking total introns dataframe 
introns_Hv_MorexHC.head()

In [ ]:
### --- Checking UTR introns dataframe
introns_UTR_Hv_MorexHC.head()

In [ ]:
### --- Function to create remove from the total introns the one within UTRs and create a dataframe with introns within cds
### --- The indataframe must be formatted like introns_Hv_MorexHC
### --- The assumption is to use pd.concat() to concatenate the two dataframes and then drop the duplicates
def cds_introns_dataframe(indataframe1, indataframe2):
    total_introns_df = pd.concat([indataframe1,indataframe2])
    #print(len(total_introns_df))
    #print(len(introns_UTR))
    #print(len(introns_df))
    #print(len(total_introns_df.drop_duplicates(['ID','start','end'],keep=False))) 
    introns_cds_df = total_introns_df.drop_duplicates(['ID','start','end'],keep=False)
    return introns_cds_df

In [ ]:
### --- Extracting introns from cds within Hv_MorexHC
introns_cds_Hv_MorexHC = cds_introns_dataframe(introns_Hv_MorexHC, introns_UTR_Hv_MorexHC)
introns_cds_Hv_MorexHC

In [ ]:
### --- Checking if there are not introns UTR within the created dataframe
introns_cds_Hv_MorexHC.loc[introns_cds_Hv_MorexHC['type'].isin(['intron three_prime_UTR','intron five_prime_UTR'])]

In [ ]:
### --- Basic statistics of Hv_MorexHC introns cds
introns_cds_Hv_MorexHC['length'].describe()

In [ ]:
### --- Plotting the distribution of Hv_MorexHC cds introns length
introns_cds_Hv_MorexHC['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Hv_MorexHC cds introns length without outliers
introns_cds_Hv_MorexHC.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 2.1.6 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Hv_Morex_longread

In [ ]:
! mv Data/Hv_MorexHC* Data/Hv_Morex_longread/

In [ ]:
! ls Data/Hv_Morex_longread/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Hv_Morex_longread

In [ ]:
! mv Output/Hv_MorexHC* Output/Hv_Morex_longread

In [ ]:
! ls Output/Hv_Morex_longread/

# 2.2 Reproducing on Hv_Morex LC

### 2.2.1 Collecting Data

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Hv_Morex.pgsb.Jul2020.LC.gff3 https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/e27077bd-fa0b-4c20-ba87-5c84b9d0641c/1/DOWNLOAD

### 2.2.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Hv_Morex.pgsb.Jul2020.LC.gff3  > Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Hv_MorexLC = pd.read_csv('Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Hv_MorexLC.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Hv_MorexLC = df_Hv_MorexLC.loc[df_Hv_MorexLC['type'].isin(['exon'])]
exon_Hv_MorexLC.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Hv_MorexLC.to_csv('Data/Hv_MorexLC_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Hv_MorexLC = df_Hv_MorexLC.loc[df_Hv_MorexLC['strand'].isin(['+'])]
forw_Hv_MorexLC.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Hv_MorexLC = df_Hv_MorexLC.loc[df_Hv_MorexLC['strand'].isin(['-'])]
rev_Hv_MorexLC.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Hv_MorexLC = forw_Hv_MorexLC.loc[df_Hv_MorexLC['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Hv_MorexLC.head()

In [ ]:
### --- Writing to a tsv formatted file the UTR_df_forw dataframe
UTR_forw_Hv_MorexLC.to_csv('Data/Hv_MorexLC_UTRforw.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Hv_MorexLC = rev_Hv_MorexLC.loc[df_Hv_MorexLC['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Hv_MorexLC.head()

In [ ]:
### --- Writing to a tsv formatted file the UTR_df_rev dataframe
UTR_rev_Hv_MorexLC.to_csv('Data/Hv_MorexLC_UTRrev.tsv',sep='\t',index=False,header=False)

### 2.2.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Hv_MorexLC
extract_tot_introns('Data/Hv_MorexLC_exon.tsv', 'Output/Hv_MorexLC_introns.tsv')

In [ ]:
introns_Hv_MorexLC = pd.read_csv('Output/Hv_MorexLC_introns.tsv', sep = '\t')
introns_Hv_MorexLC[:100]

In [ ]:
### --- Counting introns type in Hv_MorexHC
introns_dictionary_Hv_MorexLC = counting_introns_type(exon_Hv_MorexLC)

In [ ]:
n_intronless_Hv_MorexLC, n_intronpoor_Hv_MorexLC, n_intronrich_Hv_MorexLC, intronless_Hv_MorexLC, intronpoor_Hv_MorexLC, intronrich_Hv_MorexLC = splitting_introns_type(introns_dictionary_Hv_MorexLC)

In [ ]:
data_Hv_MorexLC = [34638, 9345, 1877]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Hv_MorexLC, labels = genes, autopct='%1.0f%%')
plt.title("Genes Hv_MorexLC Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Hv_MorexLC gene IDs to check the number of transcripts using command line
intronless_file_Hv_Morex_LC = open('Output/Hv_MorexLC_intronlessIDs.txt', 'a')
intronpoor_file_Hv_Morex_LC = open('Output/Hv_MorexLC_intronpoorIDs.txt', 'a')
intronrich_file_Hv_Morex_LC = open('Output/Hv_MorexLC_intronrichIDs.txt', 'a')
for i in range(len(intronless_Hv_MorexLC)):
    if i == len(intronless_Hv_MorexLC) -1:
        intronless_file_Hv_Morex_LC.write(intronless_Hv_MorexLC[i])
    else:
        intronless_file_Hv_Morex_LC.write(intronless_Hv_MorexLC[i] + '\n')
intronless_file_Hv_Morex_LC.close()

for i in range(len(intronpoor_Hv_MorexLC)):
    if i == len(intronpoor_Hv_MorexLC) -1:
        intronpoor_file_Hv_Morex_LC.write(intronpoor_Hv_MorexLC[i])
    else:
        intronpoor_file_Hv_Morex_LC.write(intronpoor_Hv_MorexLC[i] + '\n')
intronpoor_file_Hv_Morex_LC.close()

for i in range(len(intronrich_Hv_MorexLC)):
    if i == len(intronrich_Hv_MorexLC) -1:
        intronrich_file_Hv_Morex_LC.write(intronrich_Hv_MorexLC[i])
    else:
        intronrich_file_Hv_Morex_LC.write(intronrich_Hv_MorexLC[i]+'\n')
intronrich_file_Hv_Morex_LC.close()


In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Hv_MorexLC = ! grep -f Output/Hv_MorexLC_intronlessIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Hv_MorexLC = ! grep -f Output/Hv_MorexLC_intronpoorIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Hv_MorexLC = ! grep -f Output/Hv_MorexLC_intronrichIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.LC.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Hv_MorexLC = int(n_mRNA_intronless_Hv_MorexLC[0])/n_intronless_Hv_MorexLC
print(avg_mRNA_intronless_Hv_MorexLC)
avg_mRNA_intronpoor_Hv_MorexLC = int(n_mRNA_intronpoor_Hv_MorexLC[0])/n_intronpoor_Hv_MorexLC
print(avg_mRNA_intronpoor_Hv_MorexLC)
avg_mRNA_intronrich_Hv_MorexLC = int(n_mRNA_intronrich_Hv_MorexLC[0])/n_intronrich_Hv_MorexLC
print(avg_mRNA_intronrich_Hv_MorexLC)

### 2.2.4 Extracting introns from UTR regions

In [ ]:
### --- Extracting introns from UTR forward Hv_MorexLC
extract_UTR_introns('Data/Hv_MorexLC_UTRforw.tsv', 'Output/Hv_MorexLC_UTRforw_introns.tsv')

In [ ]:
### --- Extracting introns from UTR reverse Hv_MorexLC
extract_UTR_introns('Data/Hv_MorexLC_UTRrev.tsv', 'Output/Hv_MorexLC_UTRrev_introns.tsv')

In [ ]:
introns_UTR_forw_Hv_MorexLC = pd.read_csv('Output/Hv_MorexLC_UTRforw_introns.tsv', sep='\t')
introns_UTR_forw_Hv_MorexLC.head()

In [ ]:
introns_UTR_rev_Hv_MorexLC = pd.read_csv('Output/Hv_MorexLC_UTRrev_introns.tsv', sep = '\t')
introns_UTR_rev_Hv_MorexLC.head()

In [ ]:
### --- Concatenating the UTR_introns to build a unique dataframe
frames_Hv_MorexLC = [introns_UTR_forw_Hv_MorexLC, introns_UTR_rev_Hv_MorexLC]
introns_UTR_Hv_MorexLC = pd.concat(frames_Hv_MorexLC)

In [ ]:
### --- Performing some basic statistics 
introns_UTR_Hv_MorexLC['length'].describe()

In [ ]:
### --- Plotting the distribution of UTR introns length
introns_UTR_Hv_MorexLC['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of UTR introns length without outliers
introns_UTR_Hv_MorexLC.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

In [ ]:
### --- splitting in introns 3'UTR and 5'UTR to eventually if there are differences among the two
introns_UTR3_Hv_MorexLC = introns_UTR_Hv_MorexLC.loc[introns_UTR_Hv_MorexLC['type'].isin(['intron three_prime_UTR'])]
introns_UTR5_Hv_MorexLC = introns_UTR_Hv_MorexLC.loc[introns_UTR_Hv_MorexLC['type'].isin(['intron five_prime_UTR'])]

In [ ]:
introns_UTR3_Hv_MorexLC

In [ ]:
introns_UTR3_Hv_MorexLC['length'].describe()

In [ ]:
introns_UTR5_Hv_MorexLC

In [ ]:
introns_UTR5_Hv_MorexLC['length'].describe()

In [ ]:
### --- Plotting the distribution of introns_UTR vs total UTRs
labels_Hv_MorexLC = ['UTRs', 'Introns_5UTR', 'Introns_3UTR']
UTR_data_Hv_MorexLC = [ 1800+1964-(282+380), 282, 380]
fig = plt.figure(figsize =(10, 7))
plt.pie(UTR_data_Hv_MorexLC, labels = labels_Hv_MorexLC, autopct='%1.0f%%')
plt.title("UTRs Hv_MorexLC Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

### 2.2.5 Removing UTR introns from the whole introns dataframe

In [ ]:
### --- Checking total introns dataframe 
introns_Hv_MorexLC.head()

In [ ]:
### --- Checking UTR introns dataframe
introns_UTR_Hv_MorexLC.head()

In [ ]:
### --- Extracting introns from cds within Hv_MorexHC
introns_cds_Hv_MorexLC = cds_introns_dataframe(introns_Hv_MorexLC, introns_UTR_Hv_MorexLC)
introns_cds_Hv_MorexLC

In [ ]:
### --- Checking if there are not introns UTR within the created dataframe
introns_cds_Hv_MorexLC.loc[introns_cds_Hv_MorexLC['type'].isin(['intron three_prime_UTR','intron five_prime_UTR'])]

In [ ]:
### --- Basic statistics of Hv_MorexLC introns cds
introns_cds_Hv_MorexLC['length'].describe()

In [ ]:
### --- Plotting the distribution of Hv_MorexLC cds introns length
introns_cds_Hv_MorexLC['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Hv_MorexLC cds introns length without outliers
introns_cds_Hv_MorexLC.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 2.2.6 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mv Data/Hv_MorexLC* Data/Hv_Morex_longread/

In [ ]:
! ls Data/Hv_Morex_longread/

In [ ]:
! ls Output/

In [ ]:
! mv Output/Hv_MorexLC* Output/Hv_Morex_longread/

In [ ]:
! ls Output/Hv_Morex_longread/

# 2.3 Reproducing on Hv_Morex total

### 2.3.1 Collecting Data

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Hv_Morex.pgsb.Jul2020.gff3 https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/5d16cc17-c37f-417f-855d-c5e72c721f6c/1/DOWNLOAD 

### 2.3.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Hv_Morex.pgsb.Jul2020.gff3  > Data/Hv_Morex_nohashtag.pgsb.Jul2020.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Hv_Morex_nohashtag.pgsb.Jul2020.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Hv_Morex_nohashtag.pgsb.Jul2020.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Hv_Morex = pd.read_csv('Data/Hv_Morex_nohashtag.pgsb.Jul2020.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Hv_Morex.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Hv_Morex = df_Hv_Morex.loc[df_Hv_Morex['type'].isin(['exon'])]
exon_Hv_Morex.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Hv_Morex.to_csv('Data/Hv_Morex_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Hv_Morex = df_Hv_Morex.loc[df_Hv_Morex['strand'].isin(['+'])]
forw_Hv_Morex.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Hv_Morex = df_Hv_Morex.loc[df_Hv_Morex['strand'].isin(['-'])]
rev_Hv_Morex.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Hv_Morex = forw_Hv_Morex.loc[df_Hv_Morex['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Hv_Morex.head()

In [ ]:
### --- Writing to a tsv formatted file the UTR_df_rev dataframe
UTR_forw_Hv_Morex.to_csv('Data/Hv_Morex_UTRforw.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Hv_Morex = rev_Hv_Morex.loc[df_Hv_Morex['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Hv_Morex.head()

In [ ]:
### --- Writing to a tsv formatted file the UTR_df_rev dataframe
UTR_rev_Hv_Morex.to_csv('Data/Hv_Morex_UTRrev.tsv',sep='\t',index=False,header=False)

### 2.3.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Hv_Morex
extract_tot_introns('Data/Hv_Morex_exon.tsv', 'Output/Hv_Morex_introns.tsv')

In [ ]:
introns_Hv_Morex = pd.read_csv('Output/Hv_Morex_introns.tsv', sep = '\t')
introns_Hv_Morex[:100]

In [ ]:
### --- Counting introns type in Hv_MorexHC
introns_dictionary_Hv_Morex = counting_introns_type(exon_Hv_Morex)

In [ ]:
n_intronless_Hv_Morex, n_intronpoor_Hv_Morex, n_intronrich_Hv_Morex, intronless_Hv_Morex, intronpoor_Hv_Morex, intronrich_Hv_Morex = splitting_introns_type(introns_dictionary_Hv_Morex)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Hv_Morex = [46175, 22342, 13170]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Hv_Morex, labels = genes, autopct='%1.0f%%')
plt.title("Genes Hv_Morex Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Hv_Morex gene IDs to check the number of transcripts using command line
intronless_file_Hv_Morex = open('Output/Hv_Morex_intronlessIDs.txt', 'a')
intronpoor_file_Hv_Morex = open('Output/Hv_Morex_intronpoorIDs.txt', 'a')
intronrich_file_Hv_Morex = open('Output/Hv_Morex_intronrichIDs.txt', 'a')
for i in range(len(intronless_Hv_Morex)):
    if i == len(intronless_Hv_Morex) -1:
        intronless_file_Hv_Morex.write(intronless_Hv_Morex[i])
    else:
        intronless_file_Hv_Morex.write(intronless_Hv_Morex[i] + '\n')
intronless_file_Hv_Morex.close()

for i in range(len(intronpoor_Hv_Morex)):
    if i == len(intronpoor_Hv_Morex) -1:
        intronpoor_file_Hv_Morex.write(intronpoor_Hv_Morex[i])
    else:
        intronpoor_file_Hv_Morex.write(intronpoor_Hv_Morex[i] + '\n')
intronpoor_file_Hv_Morex.close()

for i in range(len(intronrich_Hv_Morex)):
    if i == len(intronrich_Hv_Morex) -1:
        intronrich_file_Hv_Morex.write(intronrich_Hv_Morex[i])
    else:
        intronrich_file_Hv_Morex.write(intronrich_Hv_Morex[i]+'\n')
intronrich_file_Hv_Morex.close()


In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Hv_Morex = ! grep -f Output/Hv_Morex_intronlessIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Hv_Morex = ! grep -f Output/Hv_Morex_intronpoorIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Hv_Morex = ! grep -f Output/Hv_Morex_intronrichIDs.txt Data/Hv_Morex_nohashtag.pgsb.Jul2020.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Hv_Morex = int(n_mRNA_intronless_Hv_Morex[0])/n_intronless_Hv_Morex
print(avg_mRNA_intronless_Hv_Morex)
avg_mRNA_intronpoor_Hv_Morex = int(n_mRNA_intronpoor_Hv_Morex[0])/n_intronpoor_Hv_Morex
print(avg_mRNA_intronpoor_Hv_Morex)
avg_mRNA_intronrich_Hv_Morex = int(n_mRNA_intronrich_Hv_Morex[0])/n_intronrich_Hv_Morex
print(avg_mRNA_intronrich_Hv_Morex)

### 2.3.4 Extracting introns from UTR regions

In [ ]:
### --- Extracting introns from UTR forward Hv_Morex
extract_UTR_introns('Data/Hv_Morex_UTRforw.tsv', 'Output/Hv_Morex_UTRforw_introns.tsv')

In [ ]:
### --- Extracting introns from UTR reverse Hv_Morex
extract_UTR_introns('Data/Hv_Morex_UTRrev.tsv', 'Output/Hv_Morex_UTRrev_introns.tsv')

In [ ]:
introns_UTR_forw_Hv_Morex = pd.read_csv('Output/Hv_Morex_UTRforw_introns.tsv', sep='\t')
introns_UTR_forw_Hv_Morex.head()

In [ ]:
introns_UTR_rev_Hv_Morex = pd.read_csv('Output/Hv_Morex_UTRrev_introns.tsv', sep = '\t')
introns_UTR_rev_Hv_Morex.head()

In [ ]:
### --- Concatenating the UTR_introns to build a unique dataframe
frames_Hv_Morex = [introns_UTR_forw_Hv_Morex, introns_UTR_rev_Hv_Morex]
introns_UTR_Hv_Morex = pd.concat(frames_Hv_Morex)

In [ ]:
### --- Performing some basic statistics 
introns_UTR_Hv_Morex['length'].describe()

In [ ]:
### --- Plotting the distribution of UTR introns length
introns_UTR_Hv_Morex['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of UTR introns length without outliers
introns_UTR_Hv_Morex.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

In [ ]:
### --- splitting in introns 3'UTR and 5'UTR to eventually if there are differences among the two
introns_UTR3_Hv_Morex = introns_UTR_Hv_Morex.loc[introns_UTR_Hv_Morex['type'].isin(['intron three_prime_UTR'])]
introns_UTR5_Hv_Morex = introns_UTR_Hv_Morex.loc[introns_UTR_Hv_Morex['type'].isin(['intron five_prime_UTR'])]

In [ ]:
introns_UTR3_Hv_Morex

In [ ]:
introns_UTR3_Hv_Morex['length'].describe()

In [ ]:
introns_UTR5_Hv_Morex

In [ ]:
introns_UTR5_Hv_Morex['length'].describe()

In [ ]:
### --- Plotting the distribution of introns_UTR vs total UTRs
labels_Hv_Morex = ['UTRs', 'Introns_5UTR', 'Introns_3UTR']
UTR_data_Hv_Morex = [22630 + 21738 -(3699+2012), 3699, 2012]
fig = plt.figure(figsize =(10, 7))
plt.pie(UTR_data_Hv_Morex, labels = labels_Hv_Morex, autopct='%1.0f%%')
plt.title("UTRs Hv_Morex Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

### 2.3.5 Removing UTR introns from the whole introns dataframe

In [ ]:
### --- Checking total introns dataframe 
introns_Hv_Morex.head()

In [ ]:
### --- Checking UTR introns dataframe
introns_UTR_Hv_Morex.head()

In [ ]:
### --- Extracting introns from cds within Hv_MorexHC
introns_cds_Hv_Morex = cds_introns_dataframe(introns_Hv_Morex, introns_UTR_Hv_Morex)
introns_cds_Hv_Morex

In [ ]:
### --- Checking if there are not introns UTR within the created dataframe
introns_cds_Hv_Morex.loc[introns_cds_Hv_Morex['type'].isin(['intron three_prime_UTR','intron five_prime_UTR'])]

In [ ]:
### --- Basic statistics of Hv_Morex introns cds
introns_cds_Hv_Morex['length'].describe()

In [ ]:
### --- Plotting the distribution of Hv_Morex cds introns length
introns_cds_Hv_Morex['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Hv_Morex cds introns length without outliers
introns_cds_Hv_Morex.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 2.3.6 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mv Data/*tsv Data/Hv_Morex_longread

In [ ]:
! mv Data/Hv_Morex_n* Data/Hv_Morex_longread/

In [ ]:
! ls Data/Hv_Morex_longread/

In [ ]:
! ls Output/

In [ ]:
! mv Output/*tsv Output/Hv_Morex_longread/

In [ ]:
! mv Output/*txt Output/Hv_Morex_longread/

In [ ]:
! ls Output/Hv_Morex_longread/

# 3 Reproducing the same analysis using data of a Pan Genome study

The Pan Genome study I am referring to can be found [here](https://www.nature.com/articles/s41586-020-2947-8). All the data can be found [here](https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/3490162b-3d76-4ba1-b6ee-3eaed5f6b644/2/).

I will firstly perform the analysis on the data within the _Denovo_gene_annotation_ directory and then on the ones within the _Gene_projection_ directory. 

## 3.1 Barke - De novo gene annotation 

### 3.1.1 Collecting Data

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Barke.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/1f736b6a-20f5-4e48-83c9-a308a51221ee/1/DOWNLOAD

### 3.1.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Barke.gff3  > Data/Barke_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Barke_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Barke_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Barke = pd.read_csv('Data/Barke_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Barke.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Barke = df_Barke.loc[df_Barke['type'].isin(['exon'])]
exon_Barke.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Barke.to_csv('Data/Barke_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Barke = df_Barke.loc[df_Barke['strand'].isin(['+'])]
forw_Barke.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Barke = df_Barke.loc[df_Barke['strand'].isin(['-'])]
rev_Barke.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Barke = forw_Barke.loc[df_Barke['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Barke.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Barke = rev_Barke.loc[df_Barke['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Barke.head()

As we see UTRs are not reported within this annotation so we cannot analyze them. We will just focus on the total number of introns. 

### 3.1.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Barke
extract_tot_introns('Data/Barke_exon.tsv', 'Output/Barke_introns.tsv')

In [ ]:
introns_Barke = pd.read_csv('Output/Barke_introns.tsv', sep = '\t')
introns_Barke[:100]

In [ ]:
### --- Counting introns type in Barke
introns_dictionary_Barke = counting_introns_type(exon_Barke)

In [ ]:
n_intronless_Barke, n_intronpoor_Barke, n_intronrich_Barke, intronless_Barke, intronpoor_Barke, intronrich_Barke = splitting_introns_type(introns_dictionary_Barke)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Barke = [25166, 14852, 11019]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Barke, labels = genes, autopct='%1.0f%%')
plt.title("Genes Barke Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Barke gene IDs to check the number of transcripts using command line
intronless_file_Barke = open('Output/Barke_intronlessIDs.txt', 'a')
intronpoor_file_Barke = open('Output/Barke_intronpoorIDs.txt', 'a')
intronrich_file_Barke = open('Output/Barke_intronrichIDs.txt', 'a')
for i in range(len(intronless_Barke)):
    if i == len(intronless_Barke) -1:
        intronless_file_Barke.write(intronless_Barke[i])
    else:
        intronless_file_Barke.write(intronless_Barke[i] + '\n')
intronless_file_Barke.close()

for i in range(len(intronpoor_Barke)):
    if i == len(intronpoor_Barke) -1:
        intronpoor_file_Barke.write(intronpoor_Barke[i])
    else:
        intronpoor_file_Barke.write(intronpoor_Barke[i] + '\n')
intronpoor_file_Barke.close()

for i in range(len(intronrich_Barke)):
    if i == len(intronrich_Barke) -1:
        intronrich_file_Barke.write(intronrich_Barke[i])
    else:
        intronrich_file_Barke.write(intronrich_Barke[i]+'\n')
intronrich_file_Barke.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Barke = ! grep -f Output/Barke_intronlessIDs.txt Data/Barke_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Barke = ! grep -f Output/Barke_intronpoorIDs.txt Data/Barke_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Barke = ! grep -f Output/Barke_intronrichIDs.txt Data/Barke_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Barke = int(n_mRNA_intronless_Barke[0])/n_intronless_Barke
print(avg_mRNA_intronless_Barke)
avg_mRNA_intronpoor_Barke = int(n_mRNA_intronpoor_Barke[0])/n_intronpoor_Barke
print(avg_mRNA_intronpoor_Barke)
avg_mRNA_intronrich_Barke = int(n_mRNA_intronrich_Barke[0])/n_intronrich_Barke
print(avg_mRNA_intronrich_Barke)

It seems there are no transcripts undergoing alternative splicing. This is probably due to a poor annotation. 

In [ ]:
### --- Basic statistics of Barke introns
introns_Barke['length'].describe()

In [ ]:
### --- Plotting the distribution of Barke introns length
introns_Barke['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Barke introns length without outliers
introns_Barke.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.1.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Barke

In [ ]:
! mv Data/Barke_* Data/Barke/ 

In [ ]:
! ls Data/Barke/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Barke

In [ ]:
! mv Output/Barke_* Output/Barke/

In [ ]:
! ls Output/Barke/

## 3.2 HOR10350 - De novo gene annotation

### 3.2.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR10350.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/4435ac3e-52f1-47df-9709-4c30e4d21131/1/DOWNLOAD

### 3.2.2 Processing Data

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR10350.gff3  > Data/HOR10350_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR10350_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR10350_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR10350 = pd.read_csv('Data/HOR10350_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR10350.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR10350 = df_HOR10350.loc[df_HOR10350['type'].isin(['exon'])]
exon_HOR10350.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR10350.to_csv('Data/HOR10350_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR10350 = df_HOR10350.loc[df_HOR10350['strand'].isin(['+'])]
forw_HOR10350.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR10350 = df_HOR10350.loc[df_HOR10350['strand'].isin(['-'])]
rev_HOR10350.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR10350 = forw_HOR10350.loc[df_HOR10350['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR10350.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR10350 = rev_HOR10350.loc[df_HOR10350['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR10350.head()

As before there are no UTRs due to a poor annotation. 

### 3.2.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR10350
extract_tot_introns('Data/HOR10350_exon.tsv', 'Output/HOR10350_introns.tsv')

In [ ]:
introns_HOR10350 = pd.read_csv('Output/HOR10350_introns.tsv', sep = '\t')
introns_HOR10350[:100]

In [ ]:
### --- Counting introns type in HOR10350
introns_dictionary_HOR10350 = counting_introns_type(exon_HOR10350)

In [ ]:
n_intronless_HOR10350, n_intronpoor_HOR10350, n_intronrich_HOR10350, intronless_HOR10350, intronpoor_HOR10350, intronrich_HOR10350 = splitting_introns_type(introns_dictionary_HOR10350)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_HOR10350 = [24866, 14709, 11126]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_HOR10350, labels = genes, autopct='%1.0f%%')
plt.title("Genes HOR10350 Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of HOR10350 gene IDs to check the number of transcripts using command line
intronless_file_HOR10350 = open('Output/HOR10350_intronlessIDs.txt', 'a')
intronpoor_file_HOR10350 = open('Output/HOR10350_intronpoorIDs.txt', 'a')
intronrich_file_HOR10350 = open('Output/HOR10350_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR10350)):
    if i == len(intronless_HOR10350) -1:
        intronless_file_HOR10350.write(intronless_HOR10350[i])
    else:
        intronless_file_HOR10350.write(intronless_HOR10350[i] + '\n')
intronless_file_HOR10350.close()

for i in range(len(intronpoor_HOR10350)):
    if i == len(intronpoor_HOR10350) -1:
        intronpoor_file_HOR10350.write(intronpoor_HOR10350[i])
    else:
        intronpoor_file_HOR10350.write(intronpoor_HOR10350[i] + '\n')
intronpoor_file_HOR10350.close()

for i in range(len(intronrich_HOR10350)):
    if i == len(intronrich_HOR10350) -1:
        intronrich_file_HOR10350.write(intronrich_HOR10350[i])
    else:
        intronrich_file_HOR10350.write(intronrich_HOR10350[i]+'\n')
intronrich_file_HOR10350.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR10350 = ! grep -f Output/HOR10350_intronlessIDs.txt Data/HOR10350_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR10350 = ! grep -f Output/HOR10350_intronpoorIDs.txt Data/HOR10350_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR10350 = ! grep -f Output/HOR10350_intronrichIDs.txt Data/HOR10350_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR10350 = int(n_mRNA_intronless_HOR10350[0])/n_intronless_HOR10350
print(avg_mRNA_intronless_HOR10350)
avg_mRNA_intronpoor_HOR10350 = int(n_mRNA_intronpoor_HOR10350[0])/n_intronpoor_HOR10350
print(avg_mRNA_intronpoor_HOR10350)
avg_mRNA_intronrich_HOR10350 = int(n_mRNA_intronrich_HOR10350[0])/n_intronrich_HOR10350
print(avg_mRNA_intronrich_HOR10350)

Once again no more than 1 transcript/gene annotated.

In [ ]:
### --- Basic statistics of HOR10350 introns
introns_HOR10350['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR10350 introns length
introns_HOR10350['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR10350 introns length without outliers
introns_HOR10350.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.2.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR10350

In [ ]:
! mv Data/HOR10350_* Data/HOR10350/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR10350

In [ ]:
! mv Output/HOR10350_* Output/HOR10350/

In [ ]:
! ls Output/

## 3.3 Morex - De novo gene annotation

### 3.3.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Morex.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/f937d01e-f9cf-415a-961b-c490db136f92/1/DOWNLOAD

### 3.3.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Morex.gff3  > Data/Morex_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Morex_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Morex_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Morex = pd.read_csv('Data/Morex_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Morex.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Morex = df_Morex.loc[df_Morex['type'].isin(['exon'])]
exon_Morex.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Morex.to_csv('Data/Morex_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Morex = df_Morex.loc[df_Morex['strand'].isin(['+'])]
forw_Morex.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Morex = df_Morex.loc[df_Morex['strand'].isin(['-'])]
rev_Morex.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Morex = forw_Morex.loc[df_Morex['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Morex.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Morex = rev_Morex.loc[df_Morex['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Morex.head()

As in the other cases no UTRs annotated.

### 3.3.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Morex
extract_tot_introns('Data/Morex_exon.tsv', 'Output/Morex_introns.tsv')

In [ ]:
introns_Morex = pd.read_csv('Output/Morex_introns.tsv', sep = '\t')
introns_Morex[:100]

In [ ]:
### --- Counting introns type in Morex
introns_dictionary_Morex = counting_introns_type(exon_Morex)

In [ ]:
n_intronless_Morex, n_intronpoor_Morex, n_intronrich_Morex, intronless_Morex, intronpoor_Morex, intronrich_Morex = splitting_introns_type(introns_dictionary_Morex)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Morex = [33663, 17811, 12184]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Morex, labels = genes, autopct='%1.0f%%')
plt.title("Genes Morex Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Morex gene IDs to check the number of transcripts using command line
intronless_file_Morex = open('Output/Morex_intronlessIDs.txt', 'a')
intronpoor_file_Morex = open('Output/Morex_intronpoorIDs.txt', 'a')
intronrich_file_Morex = open('Output/Morex_intronrichIDs.txt', 'a')
for i in range(len(intronless_Morex)):
    if i == len(intronless_Morex) -1:
        intronless_file_Morex.write(intronless_Morex[i])
    else:
        intronless_file_Morex.write(intronless_Morex[i] + '\n')
intronless_file_Morex.close()

for i in range(len(intronpoor_Morex)):
    if i == len(intronpoor_Morex) -1:
        intronpoor_file_Morex.write(intronpoor_Morex[i])
    else:
        intronpoor_file_Morex.write(intronpoor_Morex[i] + '\n')
intronpoor_file_Morex.close()

for i in range(len(intronrich_Morex)):
    if i == len(intronrich_Morex) -1:
        intronrich_file_Morex.write(intronrich_Morex[i])
    else:
        intronrich_file_Morex.write(intronrich_Morex[i]+'\n')
intronrich_file_Morex.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Morex = ! grep -f Output/Morex_intronlessIDs.txt Data/Morex_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Morex = ! grep -f Output/Morex_intronpoorIDs.txt Data/Morex_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Morex = ! grep -f Output/Morex_intronrichIDs.txt Data/Morex_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Morex = int(n_mRNA_intronless_Morex[0])/n_intronless_Morex
print(avg_mRNA_intronless_Morex)
avg_mRNA_intronpoor_Morex = int(n_mRNA_intronpoor_Morex[0])/n_intronpoor_Morex
print(avg_mRNA_intronpoor_Morex)
avg_mRNA_intronrich_Morex = int(n_mRNA_intronrich_Morex[0])/n_intronrich_Morex
print(avg_mRNA_intronrich_Morex)

Poor annotation once again. 

In [ ]:
### --- Basic statistics of Morex introns
introns_Morex['length'].describe()

In [ ]:
### --- Plotting the distribution of Morex introns length
introns_Morex['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Morex introns length without outliers
introns_Morex.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.3.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Morex

In [ ]:
! mv Data/Morex_* Data/Morex/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Morex

In [ ]:
! mv Output/Morex_* Output/Morex/

In [ ]:
! ls Output/

## 3.4 Re-defining the functions due to a different 9th field format of the genomes within the Gene_projection directory

In [ ]:
### --- Function to extract introns from the whole genome
### --- Infile is a file like Akashinriki_exon.tsv 
### --- Outfile will be written thanks to this function
### --- If there are two or more exons belonging to the same mRNA one after the other it compute the introns separating them
def extract_tot_introns_gp(infile, outfile):
    introns = open(outfile, 'a') 
    introns.write('ID'+'\t'+'type'+'\t'+'start'+'\t'+'end'+'\t'+'length') 
    with open(infile) as f: 
        lines = f.readlines()
        for i in range(0, len(lines)):
            if i+1 == len(lines): # This is to avoid out of range error
                break 
            else:
                line = lines[i]
                line = line.rstrip()
                line = line.split()
                next_line = lines[i+1]
                next_line = lines[i+1].rstrip()
                next_line = lines[i+1].split()
                if '_'.join(line[8].split('_')[0:3]) != '_'.join(next_line[8].split('_')[0:3]): continue # Checking if the next exon has the same ID of the one we are at
                
                introns.write('\n'+'_'.join(line[8].split('_')[0:3])+'\t'+ 'intron ' +'\t'+str(int(line[4])+1)+'\t'+str(int(next_line[3])-1)+'\t'+str(int(next_line[3])-1-int(line[4])+1))
    introns.close()


In [ ]:
### --- Function to count introns within a dataframe made of exons formatted like exon_Akashinriki

def counting_introns_type_gp(in_dataframe):
    counts = dict()
    for i in in_dataframe['attributes']:
        counts['_'.join(i.split('_')[0:3])] = counts.get('_'.join(i.split('_')[0:3]), 0) + 1
    return counts

In [ ]:
### --- Function to extract introns from UTR regions.
### --- Infile is a file like Akashinriki_UTRforw_introns.tsv 
### --- Outfile will be written thanks to this function
### --- Being all on the same strand when two or more UTRs of the same type are one row after the other it means there is a intron separating them
def extract_UTR_introns_gp(infile, outfile):
    UTR_introns_forw = open(outfile, 'a')
    UTR_introns_forw.write('ID'+'\t'+'type'+'\t'+'start'+'\t'+'end'+'\t'+'length') # Defining the header
    with open(infile) as f:
        lines = f.readlines()
        for i in range(0, len(lines)):
            if i+1 == len(lines):
                break
            else:
                line = lines[i]
                line = line.rstrip()
                line = line.split()
                next_line = lines[i+1]
                next_line = lines[i+1].rstrip()
                next_line = lines[i+1].split()
                if line[2] != next_line[2] or '_'.join(line[8].split('_')[0:3]) != '_'.join(next_line[8].split('_')[0:3]): continue
                
                UTR_introns_forw.write('\n'+'_'.join(line[8].split('_')[0:3])+'\t'+ 'intron '+line[2]+'\t'+str(int(line[4])+1)+'\t'+str(int(next_line[3])-1)+'\t'+str(int(next_line[3])-1-int(line[4])+1))
    UTR_introns_forw.close()

## 3.5 Akashinriki - Gene projection

### 3.5.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Akashinriki.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/325492c0-89ef-4beb-87ef-c2372093a918/1/DOWNLOAD

### 3.5.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Akashinriki.gff3  > Data/Akashinriki_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Akashinriki_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Akashinriki_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Akashinriki = pd.read_csv('Data/Akashinriki_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Akashinriki.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Akashinriki = df_Akashinriki.loc[df_Akashinriki['type'].isin(['exon'])]
exon_Akashinriki.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Akashinriki.to_csv('Data/Akashinriki_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Akashinriki = df_Akashinriki.loc[df_Akashinriki['strand'].isin(['+'])]
forw_Akashinriki.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Akashinriki = df_Akashinriki.loc[df_Akashinriki['strand'].isin(['-'])]
rev_Akashinriki.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Akashinriki = forw_Akashinriki.loc[df_Akashinriki['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Akashinriki.head()


In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Akashinriki = rev_Akashinriki.loc[df_Akashinriki['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Akashinriki.head()

Still no UTRs.

### 3.5.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Akashinriki
extract_tot_introns_gp('Data/Akashinriki_exon.tsv', 'Output/Akashinriki_introns.tsv')

In [ ]:
introns_Akashinriki = pd.read_csv('Output/Akashinriki_introns.tsv', sep = '\t')
introns_Akashinriki[:100]

In [ ]:
### --- Counting introns type in Akashinriki
introns_dictionary_Akashinriki = counting_introns_type_gp(exon_Akashinriki)

In [ ]:
n_intronless_Akashinriki, n_intronpoor_Akashinriki, n_intronrich_Akashinriki, intronless_Akashinriki, intronpoor_Akashinriki, intronrich_Akashinriki = splitting_introns_type(introns_dictionary_Akashinriki)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Akashinriki = [16635, 18474, 12050]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Akashinriki, labels = genes, autopct='%1.0f%%')
plt.title("Genes Akashinriki Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Akashinriki gene IDs to check the number of transcripts using command line
intronless_file_Akashinriki = open('Output/Akashinriki_intronlessIDs.txt', 'a')
intronpoor_file_Akashinriki = open('Output/Akashinriki_intronpoorIDs.txt', 'a')
intronrich_file_Akashinriki = open('Output/Akashinriki_intronrichIDs.txt', 'a')
for i in range(len(intronless_Akashinriki)):
    if i == len(intronless_Akashinriki) -1:
        intronless_file_Akashinriki.write(intronless_Akashinriki[i])
    else:
        intronless_file_Akashinriki.write(intronless_Akashinriki[i] + '\n')
intronless_file_Akashinriki.close()

for i in range(len(intronpoor_Akashinriki)):
    if i == len(intronpoor_Akashinriki) -1:
        intronpoor_file_Akashinriki.write(intronpoor_Akashinriki[i])
    else:
        intronpoor_file_Akashinriki.write(intronpoor_Akashinriki[i] + '\n')
intronpoor_file_Akashinriki.close()

for i in range(len(intronrich_Akashinriki)):
    if i == len(intronrich_Akashinriki) -1:
        intronrich_file_Akashinriki.write(intronrich_Akashinriki[i])
    else:
        intronrich_file_Akashinriki.write(intronrich_Akashinriki[i]+'\n')
intronrich_file_Akashinriki.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Akashinriki = ! grep -f Output/Akashinriki_intronlessIDs.txt Data/Akashinriki_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Akashinriki = ! grep -f Output/Akashinriki_intronpoorIDs.txt Data/Akashinriki_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Akashinriki = ! grep -f Output/Akashinriki_intronrichIDs.txt Data/Akashinriki_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Akashinriki = int(n_mRNA_intronless_Akashinriki[0])/n_intronless_Akashinriki
print(avg_mRNA_intronless_Akashinriki)
avg_mRNA_intronpoor_Akashinriki = int(n_mRNA_intronpoor_Akashinriki[0])/n_intronpoor_Akashinriki
print(avg_mRNA_intronpoor_Akashinriki)
avg_mRNA_intronrich_Akashinriki = int(n_mRNA_intronrich_Akashinriki[0])/n_intronrich_Akashinriki
print(avg_mRNA_intronrich_Akashinriki)

Still no alternative splicing annotated.

In [ ]:
### --- Basic statistics of Akashinriki introns
introns_Akashinriki['length'].describe()

In [ ]:
### --- Plotting the distribution of Akashinriki introns length
introns_Akashinriki['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Akashinriki introns length without outliers
introns_Akashinriki.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.5.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Akashinriki 

In [ ]:
! mv Data/Akashinriki_* Data/Akashinriki/

In [ ]:
! ls Data/

In [ ]:
! ls Output/ 

In [ ]:
! mkdir Output/Akashinriki

In [ ]:
! mv Output/Akashinriki_* Output/Akashinriki/

In [ ]:
! ls Output/

## 3.6 B1K_04_12 - Gene projection

### 3.6.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/B1K.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/0ae99eec-5c46-41b6-83ab-f7f4b4e2f86c/1/DOWNLOAD

### 3.6.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/B1K.gff3  > Data/B1K_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/B1K_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/B1K_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_B1K = pd.read_csv('Data/B1K_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_B1K.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_B1K = df_B1K.loc[df_B1K['type'].isin(['exon'])]
exon_B1K.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_B1K.to_csv('Data/B1K_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_B1K = df_B1K.loc[df_B1K['strand'].isin(['+'])]
forw_B1K.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_B1K = df_B1K.loc[df_B1K['strand'].isin(['-'])]
rev_B1K.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_B1K = forw_B1K.loc[df_B1K['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_B1K.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_B1K = rev_B1K.loc[df_B1K['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_B1K.head()

No UTRs.

### 3.6.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from B1K
extract_tot_introns_gp('Data/B1K_exon.tsv', 'Output/B1K_introns.tsv')

In [ ]:
introns_B1K = pd.read_csv('Output/B1K_introns.tsv', sep = '\t')
introns_B1K[:100]

In [ ]:
### --- Counting introns type in B1K
introns_dictionary_B1K = counting_introns_type_gp(exon_B1K)

In [ ]:
n_intronless_B1K, n_intronpoor_B1K, n_intronrich_B1K, intronless_B1K, intronpoor_B1K, intronrich_B1K = splitting_introns_type(introns_dictionary_B1K)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_B1K = [17002, 18551, 12034]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_B1K, labels = genes, autopct='%1.0f%%')
plt.title("Genes B1K Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of B1K gene IDs to check the number of transcripts using command line
intronless_file_B1K = open('Output/B1K_intronlessIDs.txt', 'a')
intronpoor_file_B1K = open('Output/B1K_intronpoorIDs.txt', 'a')
intronrich_file_B1K = open('Output/B1K_intronrichIDs.txt', 'a')
for i in range(len(intronless_B1K)):
    if i == len(intronless_B1K) -1:
        intronless_file_B1K.write(intronless_B1K[i])
    else:
        intronless_file_B1K.write(intronless_B1K[i] + '\n')
intronless_file_B1K.close()

for i in range(len(intronpoor_B1K)):
    if i == len(intronpoor_B1K) -1:
        intronpoor_file_B1K.write(intronpoor_B1K[i])
    else:
        intronpoor_file_B1K.write(intronpoor_B1K[i] + '\n')
intronpoor_file_B1K.close()

for i in range(len(intronrich_B1K)):
    if i == len(intronrich_B1K) -1:
        intronrich_file_B1K.write(intronrich_B1K[i])
    else:
        intronrich_file_B1K.write(intronrich_B1K[i]+'\n')
intronrich_file_B1K.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_B1K = ! grep -f Output/B1K_intronlessIDs.txt Data/B1K_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_B1K = ! grep -f Output/B1K_intronpoorIDs.txt Data/B1K_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_B1K = ! grep -f Output/B1K_intronrichIDs.txt Data/B1K_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_B1K = int(n_mRNA_intronless_B1K[0])/n_intronless_B1K
print(avg_mRNA_intronless_B1K)
avg_mRNA_intronpoor_B1K = int(n_mRNA_intronpoor_B1K[0])/n_intronpoor_B1K
print(avg_mRNA_intronpoor_B1K)
avg_mRNA_intronrich_B1K = int(n_mRNA_intronrich_B1K[0])/n_intronrich_B1K
print(avg_mRNA_intronrich_B1K)

In [ ]:
### --- Basic statistics of B1K introns
introns_B1K['length'].describe()

In [ ]:
### --- Plotting the distribution of B1K introns length
introns_B1K['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of B1K introns length without outliers
introns_B1K.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.6.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/B1K

In [ ]:
! mv Data/B1K_* Data/B1K/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/B1K

In [ ]:
! mv Output/B1K_* Output/B1K/

In [ ]:
! ls Output/

## 3.7 Barke - Gene projection

### 3.7.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Barke_gp.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/005e26c4-051b-4fd4-8538-2abe31706449/1/DOWNLOAD

### 3.7.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Barke_gp.gff3  > Data/Barke_gp_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Barke_gp_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Barke_gp_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Barke_gp = pd.read_csv('Data/Barke_gp_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Barke_gp.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Barke_gp = df_Barke_gp.loc[df_Barke_gp['type'].isin(['exon'])]
exon_Barke_gp.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Barke_gp.to_csv('Data/Barke_gp_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Barke_gp = df_Barke_gp.loc[df_Barke_gp['strand'].isin(['+'])]
forw_Barke_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Barke_gp = df_Barke_gp.loc[df_Barke_gp['strand'].isin(['-'])]
rev_Barke_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Barke_gp = forw_Barke_gp.loc[df_Barke_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Barke_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Barke_gp = rev_Barke_gp.loc[df_Barke_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Barke_gp.head()

No UTRs.

### 3.7.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Barke_gp
extract_tot_introns_gp('Data/Barke_gp_exon.tsv', 'Output/Barke_gp_introns.tsv')

In [ ]:
introns_Barke_gp = pd.read_csv('Output/Barke_gp_introns.tsv', sep = '\t')
introns_Barke_gp[:100]

In [ ]:
### --- Counting introns type in Barke_gp
introns_dictionary_Barke_gp = counting_introns_type_gp(exon_Barke_gp)

In [ ]:
n_intronless_Barke_gp, n_intronpoor_Barke_gp, n_intronrich_Barke_gp, intronless_Barke_gp, intronpoor_Barke_gp, intronrich_Barke_gp = splitting_introns_type(introns_dictionary_Barke_gp)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Barke_gp = [17546, 18588, 12078]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Barke_gp, labels = genes, autopct='%1.0f%%')
plt.title("Genes Barke_gp Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Barke_gp gene IDs to check the number of transcripts using command line
intronless_file_Barke_gp = open('Output/Barke_gp_intronlessIDs.txt', 'a')
intronpoor_file_Barke_gp = open('Output/Barke_gp_intronpoorIDs.txt', 'a')
intronrich_file_Barke_gp = open('Output/Barke_gp_intronrichIDs.txt', 'a')
for i in range(len(intronless_Barke_gp)):
    if i == len(intronless_Barke_gp) -1:
        intronless_file_Barke_gp.write(intronless_Barke_gp[i])
    else:
        intronless_file_Barke_gp.write(intronless_Barke_gp[i] + '\n')
intronless_file_Barke_gp.close()

for i in range(len(intronpoor_Barke_gp)):
    if i == len(intronpoor_Barke_gp) -1:
        intronpoor_file_Barke_gp.write(intronpoor_Barke_gp[i])
    else:
        intronpoor_file_Barke_gp.write(intronpoor_Barke_gp[i] + '\n')
intronpoor_file_Barke_gp.close()

for i in range(len(intronrich_Barke_gp)):
    if i == len(intronrich_Barke_gp) -1:
        intronrich_file_Barke_gp.write(intronrich_Barke_gp[i])
    else:
        intronrich_file_Barke_gp.write(intronrich_Barke_gp[i]+'\n')
intronrich_file_Barke_gp.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Barke_gp = ! grep -f Output/Barke_gp_intronlessIDs.txt Data/Barke_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Barke_gp = ! grep -f Output/Barke_gp_intronpoorIDs.txt Data/Barke_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Barke_gp = ! grep -f Output/Barke_gp_intronrichIDs.txt Data/Barke_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Barke_gp = int(n_mRNA_intronless_Barke_gp[0])/n_intronless_Barke_gp
print(avg_mRNA_intronless_Barke_gp)
avg_mRNA_intronpoor_Barke_gp = int(n_mRNA_intronpoor_Barke_gp[0])/n_intronpoor_Barke_gp
print(avg_mRNA_intronpoor_Barke_gp)
avg_mRNA_intronrich_Barke_gp = int(n_mRNA_intronrich_Barke_gp[0])/n_intronrich_Barke_gp
print(avg_mRNA_intronrich_Barke_gp)

No alternative splicing annotated.

In [ ]:
### --- Basic statistics of Barke_gp introns
introns_Barke_gp['length'].describe()

In [ ]:
### --- Plotting the distribution of Barke_gp introns length
introns_Barke_gp['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Barke_gp introns length without outliers
introns_Barke_gp.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.7.4 Re-organizing files and directories

In [ ]:
! ls Data/ 

In [ ]:
! mkdir Data/Barke_gp

In [ ]:
! mv Data/Barke_gp_* Data/Barke_gp/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [903]:
! mkdir Output/Barke_gp

In [ ]:
! mv Output/Barke_gp* Output/Barke_gp/

In [ ]:
! ls Output/

## 3.8 Golden_Promise - Gene projection

### 3.8.1 Data collection

In [ ]:

### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Golden_Promise.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/936e3aac-6846-4b46-ad7f-1432f2ed6ef8/1/DOWNLOAD

### 3.8.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Golden_Promise.gff3  > Data/Golden_Promise_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Golden_Promise_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Golden_Promise_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Golden_Promise = pd.read_csv('Data/Golden_Promise_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Golden_Promise.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Golden_Promise = df_Golden_Promise.loc[df_Golden_Promise['type'].isin(['exon'])]
exon_Golden_Promise.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Golden_Promise.to_csv('Data/Golden_Promise_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Golden_Promise = df_Golden_Promise.loc[df_Golden_Promise['strand'].isin(['+'])]
forw_Golden_Promise.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Golden_Promise = df_Golden_Promise.loc[df_Golden_Promise['strand'].isin(['-'])]
rev_Golden_Promise.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Golden_Promise = forw_Golden_Promise.loc[df_Golden_Promise['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Golden_Promise.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Golden_Promise = rev_Golden_Promise.loc[df_Golden_Promise['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Golden_Promise.head()

No UTRs.

### 3.8.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Golden_Promise
extract_tot_introns_gp('Data/Golden_Promise_exon.tsv', 'Output/Golden_Promise_introns.tsv')

In [ ]:
introns_Golden_Promise = pd.read_csv('Output/Golden_Promise_introns.tsv', sep = '\t')
introns_Golden_Promise[:100]

In [ ]:
### --- Counting introns type in Golden_Promise
introns_dictionary_Golden_Promise = counting_introns_type_gp(exon_Golden_Promise)

In [ ]:
n_intronless_Golden_Promise, n_intronpoor_Golden_Promise, n_intronrich_Golden_Promise, intronless_Golden_Promise, intronpoor_Golden_Promise, intronrich_Golden_Promise = splitting_introns_type(introns_dictionary_Golden_Promise)

In [ ]:
genes = ['Intronless', 'Intronpoor', 'Intronrich']
data_Golden_Promise = [14844, 18111, 12129]
fig = plt.figure(figsize =(10, 7))
plt.pie(data_Golden_Promise, labels = genes, autopct='%1.0f%%')
plt.title("Genes Golden_Promise Distribution", bbox={'facecolor':'0.8', 'pad':5})
plt.show()

In [ ]:
### --- Creating the list of Golden_Promise gene IDs to check the number of transcripts using command line
intronless_file_Golden_Promise = open('Output/Golden_Promise_intronlessIDs.txt', 'a')
intronpoor_file_Golden_Promise = open('Output/Golden_Promise_intronpoorIDs.txt', 'a')
intronrich_file_Golden_Promise = open('Output/Golden_Promise_intronrichIDs.txt', 'a')
for i in range(len(intronless_Golden_Promise)):
    if i == len(intronless_Golden_Promise) -1:
        intronless_file_Golden_Promise.write(intronless_Golden_Promise[i])
    else:
        intronless_file_Golden_Promise.write(intronless_Golden_Promise[i] + '\n')
intronless_file_Golden_Promise.close()

for i in range(len(intronpoor_Golden_Promise)):
    if i == len(intronpoor_Golden_Promise) -1:
        intronpoor_file_Golden_Promise.write(intronpoor_Golden_Promise[i])
    else:
        intronpoor_file_Golden_Promise.write(intronpoor_Golden_Promise[i] + '\n')
intronpoor_file_Golden_Promise.close()

for i in range(len(intronrich_Golden_Promise)):
    if i == len(intronrich_Golden_Promise) -1:
        intronrich_file_Golden_Promise.write(intronrich_Golden_Promise[i])
    else:
        intronrich_file_Golden_Promise.write(intronrich_Golden_Promise[i]+'\n')
intronrich_file_Golden_Promise.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Golden_Promise = ! grep -f Output/Golden_Promise_intronlessIDs.txt Data/Golden_Promise_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Golden_Promise = ! grep -f Output/Golden_Promise_intronpoorIDs.txt Data/Golden_Promise_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Golden_Promise = ! grep -f Output/Golden_Promise_intronrichIDs.txt Data/Golden_Promise_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Golden_Promise = int(n_mRNA_intronless_Golden_Promise[0])/n_intronless_Golden_Promise
print(avg_mRNA_intronless_Golden_Promise)
avg_mRNA_intronpoor_Golden_Promise = int(n_mRNA_intronpoor_Golden_Promise[0])/n_intronpoor_Golden_Promise
print(avg_mRNA_intronpoor_Golden_Promise)
avg_mRNA_intronrich_Golden_Promise = int(n_mRNA_intronrich_Golden_Promise[0])/n_intronrich_Golden_Promise
print(avg_mRNA_intronrich_Golden_Promise)

No alternative splicing annotated.

In [ ]:
### --- Basic statistics of Golden_Promise introns
introns_Golden_Promise['length'].describe()

In [ ]:
### --- Plotting the distribution of Golden_Promise introns length
introns_Golden_Promise['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Golden_Promise introns length without outliers
introns_Golden_Promise.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.8.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Golden_Promise

In [ ]:
! mv Data/Golden_Promise_* Data/Golden_Promise/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Golden_Promise

In [ ]:
! mv Output/Golden_Promise_* Output/Golden_Promise/

In [ ]:
! ls Output/

## 3.9 Hockett - Gene projection

### 3.9.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Hockett.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/135d31f9-c030-4e95-9c50-bc61746d2721/1/DOWNLOAD

### 3.9.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Hockett.gff3  > Data/Hockett_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Hockett_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Hockett_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Hockett = pd.read_csv('Data/Hockett_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Hockett.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Hockett = df_Hockett.loc[df_Hockett['type'].isin(['exon'])]
exon_Hockett.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Hockett.to_csv('Data/Hockett_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Hockett = df_Hockett.loc[df_Hockett['strand'].isin(['+'])]
forw_Hockett.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Hockett = df_Hockett.loc[df_Hockett['strand'].isin(['-'])]
rev_Hockett.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Hockett = forw_Hockett.loc[df_Hockett['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Hockett.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Hockett = rev_Hockett.loc[df_Hockett['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Hockett.head()

No UTRs.

### 3.9.3   Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Hockett
extract_tot_introns_gp('Data/Hockett_exon.tsv', 'Output/Hockett_introns.tsv')

In [ ]:
introns_Hockett = pd.read_csv('Output/Hockett_introns.tsv', sep = '\t')
introns_Hockett[:100]

In [ ]:
### --- Counting introns type in Hockett
introns_dictionary_Hockett = counting_introns_type_gp(exon_Hockett)

In [ ]:
n_intronless_Hockett, n_intronpoor_Hockett, n_intronrich_Hockett, intronless_Hockett, intronpoor_Hockett, intronrich_Hockett = splitting_introns_type(introns_dictionary_Hockett)

In [ ]:
### --- I decided to create a function to plot the pie representing the distribution of intronless/poor/rich
def plotting_intron_types(n_intronless, n_intronpoor, n_intronrich, title):
    genes = ['Intronless', 'Intronpoor', 'Intronrich']
    data = [n_intronless, n_intronpoor, n_intronrich]
    fig = plt.figure(figsize = (10,7))
    plt.pie(data, labels = genes, autopct='%1.0f%%')
    plt.title(title, bbox={'facecolor':'0.8', 'pad':5})
    plt.show()

In [ ]:
plotting_intron_types(n_intronless_Hockett, n_intronpoor_Hockett, n_intronrich_Hockett, 'Genes Hockett Distribution')

In [ ]:
### --- Creating the list of Hockett gene IDs to check the number of transcripts using command line
intronless_file_Hockett = open('Output/Hockett_intronlessIDs.txt', 'a')
intronpoor_file_Hockett = open('Output/Hockett_intronpoorIDs.txt', 'a')
intronrich_file_Hockett = open('Output/Hockett_intronrichIDs.txt', 'a')
for i in range(len(intronless_Hockett)):
    if i == len(intronless_Hockett) -1:
        intronless_file_Hockett.write(intronless_Hockett[i])
    else:
        intronless_file_Hockett.write(intronless_Hockett[i] + '\n')
intronless_file_Hockett.close()

for i in range(len(intronpoor_Hockett)):
    if i == len(intronpoor_Hockett) -1:
        intronpoor_file_Hockett.write(intronpoor_Hockett[i])
    else:
        intronpoor_file_Hockett.write(intronpoor_Hockett[i] + '\n')
intronpoor_file_Hockett.close()

for i in range(len(intronrich_Hockett)):
    if i == len(intronrich_Hockett) -1:
        intronrich_file_Hockett.write(intronrich_Hockett[i])
    else:
        intronrich_file_Hockett.write(intronrich_Hockett[i]+'\n')
intronrich_file_Hockett.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Hockett = ! grep -f Output/Hockett_intronlessIDs.txt Data/Hockett_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Hockett = ! grep -f Output/Hockett_intronpoorIDs.txt Data/Hockett_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Hockett = ! grep -f Output/Hockett_intronrichIDs.txt Data/Hockett_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Hockett = int(n_mRNA_intronless_Hockett[0])/n_intronless_Hockett
print(avg_mRNA_intronless_Hockett)
avg_mRNA_intronpoor_Hockett = int(n_mRNA_intronpoor_Hockett[0])/n_intronpoor_Hockett
print(avg_mRNA_intronpoor_Hockett)
avg_mRNA_intronrich_Hockett = int(n_mRNA_intronrich_Hockett[0])/n_intronrich_Hockett
print(avg_mRNA_intronrich_Hockett)

No alternative splicing annotated.

In [ ]:
### --- Basic statistics of Hockett introns
introns_Hockett['length'].describe()

In [ ]:
### --- Plotting the distribution of Hockett introns length
introns_Hockett['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Hockett introns length without outliers
introns_Hockett.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.9.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Hockett

In [ ]:
! mv Data/Hockett_* Data/Hockett/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Hockett

In [ ]:
! mv Output/Hockett_* Output/Hockett/

In [ ]:
! ls Output/

## 3.10 HOR10350 - Gene projection

### 3.10.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR10350_gp.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/a909ad41-c116-4131-bf79-6e760c2cd723/1/DOWNLOAD

### 3.10.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR10350_gp.gff3  > Data/HOR10350_gp_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR10350_gp_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR10350_gp_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR10350_gp = pd.read_csv('Data/HOR10350_gp_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR10350_gp.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR10350_gp = df_HOR10350_gp.loc[df_HOR10350_gp['type'].isin(['exon'])]
exon_HOR10350_gp.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR10350_gp.to_csv('Data/HOR10350_gp_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR10350_gp = df_HOR10350_gp.loc[df_HOR10350_gp['strand'].isin(['+'])]
forw_HOR10350_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR10350_gp = df_HOR10350_gp.loc[df_HOR10350_gp['strand'].isin(['-'])]
rev_HOR10350_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR10350_gp = forw_HOR10350_gp.loc[df_HOR10350_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR10350_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR10350_gp = rev_HOR10350_gp.loc[df_HOR10350_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR10350_gp.head()

No UTRs.

### 3.10.3 Extracting introns from the whole genome 

In [ ]:
### --- Extracting introns from HOR10350_gp
extract_tot_introns_gp('Data/HOR10350_gp_exon.tsv', 'Output/HOR10350_gp_introns.tsv')

In [ ]:
introns_HOR10350_gp = pd.read_csv('Output/HOR10350_gp_introns.tsv', sep = '\t')
introns_HOR10350_gp[:100]

In [ ]:
### --- Counting introns type in HOR10350_gp
introns_dictionary_HOR10350_gp = counting_introns_type_gp(exon_HOR10350_gp)

In [ ]:
n_intronless_HOR10350_gp, n_intronpoor_HOR10350_gp, n_intronrich_HOR10350_gp, intronless_HOR10350_gp, intronpoor_HOR10350_gp, intronrich_HOR10350_gp = splitting_introns_type(introns_dictionary_HOR10350_gp)

In [ ]:
plotting_intron_types(n_intronless_HOR10350_gp, n_intronpoor_HOR10350_gp, n_intronrich_HOR10350_gp, 'Genes HOR10350_gp Distribution')

In [ ]:
### --- Creating the list of HOR10350_gp gene IDs to check the number of transcripts using command line
intronless_file_HOR10350_gp = open('Output/HOR10350_gp_intronlessIDs.txt', 'a')
intronpoor_file_HOR10350_gp = open('Output/HOR10350_gp_intronpoorIDs.txt', 'a')
intronrich_file_HOR10350_gp = open('Output/HOR10350_gp_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR10350_gp)):
    if i == len(intronless_HOR10350_gp) -1:
        intronless_file_HOR10350_gp.write(intronless_HOR10350_gp[i])
    else:
        intronless_file_HOR10350_gp.write(intronless_HOR10350_gp[i] + '\n')
intronless_file_HOR10350_gp.close()

for i in range(len(intronpoor_HOR10350_gp)):
    if i == len(intronpoor_HOR10350_gp) -1:
        intronpoor_file_HOR10350_gp.write(intronpoor_HOR10350_gp[i])
    else:
        intronpoor_file_HOR10350_gp.write(intronpoor_HOR10350_gp[i] + '\n')
intronpoor_file_HOR10350_gp.close()

for i in range(len(intronrich_HOR10350_gp)):
    if i == len(intronrich_HOR10350_gp) -1:
        intronrich_file_HOR10350_gp.write(intronrich_HOR10350_gp[i])
    else:
        intronrich_file_HOR10350_gp.write(intronrich_HOR10350_gp[i]+'\n')
intronrich_file_HOR10350_gp.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR10350_gp = ! grep -f Output/HOR10350_gp_intronlessIDs.txt Data/HOR10350_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR10350_gp = ! grep -f Output/HOR10350_gp_intronpoorIDs.txt Data/HOR10350_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR10350_gp = ! grep -f Output/HOR10350_gp_intronrichIDs.txt Data/HOR10350_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR10350_gp = int(n_mRNA_intronless_HOR10350_gp[0])/n_intronless_HOR10350_gp
print(avg_mRNA_intronless_HOR10350_gp)
avg_mRNA_intronpoor_HOR10350_gp = int(n_mRNA_intronpoor_HOR10350_gp[0])/n_intronpoor_HOR10350_gp
print(avg_mRNA_intronpoor_HOR10350_gp)
avg_mRNA_intronrich_HOR10350_gp = int(n_mRNA_intronrich_HOR10350_gp[0])/n_intronrich_HOR10350_gp
print(avg_mRNA_intronrich_HOR10350_gp)

No alternative splicing. 

In [ ]:
### --- Basic statistics of HOR10350_gp introns
introns_HOR10350_gp['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR10350_gp introns length
introns_HOR10350_gp['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR10350_gp introns length without outliers
introns_HOR10350_gp.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.10.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR10350_gp

In [ ]:
! mv Data/HOR10350_gp_* Data/HOR10350_gp/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR10350_gp

In [ ]:
! mv Output/HOR10350_gp_* Output/HOR10350_gp/

In [ ]:
! ls Output/

## 3.11 HOR13821 - Gene projection

### 3.11.1 Data collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR13821.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/da44be98-0ede-4283-872a-817d33212ff0/1/DOWNLOAD

### 3.11.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR13821.gff3  > Data/HOR13821_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR13821_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR13821_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR13821 = pd.read_csv('Data/HOR13821_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR13821.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR13821 = df_HOR13821.loc[df_HOR13821['type'].isin(['exon'])]
exon_HOR13821.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR13821.to_csv('Data/HOR13821_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR13821 = df_HOR13821.loc[df_HOR13821['strand'].isin(['+'])]
forw_HOR13821.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR13821 = df_HOR13821.loc[df_HOR13821['strand'].isin(['-'])]
rev_HOR13821.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR13821 = forw_HOR13821.loc[df_HOR13821['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR13821.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR13821 = rev_HOR13821.loc[df_HOR13821['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR13821.head()

No UTRs.

### 3.11.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR13821
extract_tot_introns_gp('Data/HOR13821_exon.tsv', 'Output/HOR13821_introns.tsv')

In [ ]:
introns_HOR13821 = pd.read_csv('Output/HOR13821_introns.tsv', sep = '\t')
introns_HOR13821[:100]

In [ ]:
### --- Counting introns type in HOR13821
introns_dictionary_HOR13821 = counting_introns_type_gp(exon_HOR13821)

In [ ]:
n_intronless_HOR13821, n_intronpoor_HOR13821, n_intronrich_HOR13821, intronless_HOR13821, intronpoor_HOR13821, intronrich_HOR13821 = splitting_introns_type(introns_dictionary_HOR13821)

In [ ]:
plotting_intron_types(n_intronless_HOR13821, n_intronpoor_HOR13821, n_intronrich_HOR13821, 'Genes HOR13821 Distribution')

In [ ]:
### --- Creating the list of HOR13821 gene IDs to check the number of transcripts using command line
intronless_file_HOR13821 = open('Output/HOR13821_intronlessIDs.txt', 'a')
intronpoor_file_HOR13821 = open('Output/HOR13821_intronpoorIDs.txt', 'a')
intronrich_file_HOR13821 = open('Output/HOR13821_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR13821)):
    if i == len(intronless_HOR13821) -1:
        intronless_file_HOR13821.write(intronless_HOR13821[i])
    else:
        intronless_file_HOR13821.write(intronless_HOR13821[i] + '\n')
intronless_file_HOR13821.close()

for i in range(len(intronpoor_HOR13821)):
    if i == len(intronpoor_HOR13821) -1:
        intronpoor_file_HOR13821.write(intronpoor_HOR13821[i])
    else:
        intronpoor_file_HOR13821.write(intronpoor_HOR13821[i] + '\n')
intronpoor_file_HOR13821.close()

for i in range(len(intronrich_HOR13821)):
    if i == len(intronrich_HOR13821) -1:
        intronrich_file_HOR13821.write(intronrich_HOR13821[i])
    else:
        intronrich_file_HOR13821.write(intronrich_HOR13821[i]+'\n')
intronrich_file_HOR13821.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR13821 = ! grep -f Output/HOR13821_intronlessIDs.txt Data/HOR13821_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR13821 = ! grep -f Output/HOR13821_intronpoorIDs.txt Data/HOR13821_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR13821 = ! grep -f Output/HOR13821_intronrichIDs.txt Data/HOR13821_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR13821 = int(n_mRNA_intronless_HOR13821[0])/n_intronless_HOR13821
print(avg_mRNA_intronless_HOR13821)
avg_mRNA_intronpoor_HOR13821 = int(n_mRNA_intronpoor_HOR13821[0])/n_intronpoor_HOR13821
print(avg_mRNA_intronpoor_HOR13821)
avg_mRNA_intronrich_HOR13821 = int(n_mRNA_intronrich_HOR13821[0])/n_intronrich_HOR13821
print(avg_mRNA_intronrich_HOR13821)

No alternative splicing.

In [ ]:
### --- Basic statistics of HOR13821 introns
introns_HOR13821['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR13821 introns length
introns_HOR13821['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR13821 introns length without outliers
introns_HOR13821.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.11.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR13821 

In [ ]:
! mv Data/HOR13821_* Data/HOR13821/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR13821

In [ ]:
! mv Output/HOR13821_* Output/HOR13821/

In [ ]:
! ls Output/

## 3.12 HOR13942 - Gene projection

### 3.12.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR13942.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/e2980ffc-4e10-4812-a1d5-90b0b27f2b9a/1/DOWNLOAD

### 3.12.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR13942.gff3  > Data/HOR13942_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR13942_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR13942_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR13942 = pd.read_csv('Data/HOR13942_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR13942.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR13942 = df_HOR13942.loc[df_HOR13942['type'].isin(['exon'])]
exon_HOR13942.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR13942.to_csv('Data/HOR13942_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR13942 = df_HOR13942.loc[df_HOR13942['strand'].isin(['+'])]
forw_HOR13942.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR13942 = df_HOR13942.loc[df_HOR13942['strand'].isin(['-'])]
rev_HOR13942.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR13942 = forw_HOR13942.loc[df_HOR13942['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR13942.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR13942 = rev_HOR13942.loc[df_HOR13942['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR13942.head()

No UTRs.

### 3.12.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR13942
extract_tot_introns_gp('Data/HOR13942_exon.tsv', 'Output/HOR13942_introns.tsv')

In [ ]:
introns_HOR13942 = pd.read_csv('Output/HOR13942_introns.tsv', sep = '\t')
introns_HOR13942[:100]

In [ ]:
### --- Counting introns type in HOR13942
introns_dictionary_HOR13942 = counting_introns_type_gp(exon_HOR13942)

In [ ]:
n_intronless_HOR13942, n_intronpoor_HOR13942, n_intronrich_HOR13942, intronless_HOR13942, intronpoor_HOR13942, intronrich_HOR13942 = splitting_introns_type(introns_dictionary_HOR13942)

In [ ]:
plotting_intron_types(n_intronless_HOR13942, n_intronpoor_HOR13942, n_intronrich_HOR13942, 'Genes HOR13942 Distribution')

In [ ]:
### --- Creating the list of HOR13942 gene IDs to check the number of transcripts using command line
intronless_file_HOR13942 = open('Output/HOR13942_intronlessIDs.txt', 'a')
intronpoor_file_HOR13942 = open('Output/HOR13942_intronpoorIDs.txt', 'a')
intronrich_file_HOR13942 = open('Output/HOR13942_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR13942)):
    if i == len(intronless_HOR13942) -1:
        intronless_file_HOR13942.write(intronless_HOR13942[i])
    else:
        intronless_file_HOR13942.write(intronless_HOR13942[i] + '\n')
intronless_file_HOR13942.close()

for i in range(len(intronpoor_HOR13942)):
    if i == len(intronpoor_HOR13942) -1:
        intronpoor_file_HOR13942.write(intronpoor_HOR13942[i])
    else:
        intronpoor_file_HOR13942.write(intronpoor_HOR13942[i] + '\n')
intronpoor_file_HOR13942.close()

for i in range(len(intronrich_HOR13942)):
    if i == len(intronrich_HOR13942) -1:
        intronrich_file_HOR13942.write(intronrich_HOR13942[i])
    else:
        intronrich_file_HOR13942.write(intronrich_HOR13942[i]+'\n')
intronrich_file_HOR13942.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR13942 = ! grep -f Output/HOR13942_intronlessIDs.txt Data/HOR13942_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR13942 = ! grep -f Output/HOR13942_intronpoorIDs.txt Data/HOR13942_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR13942 = ! grep -f Output/HOR13942_intronrichIDs.txt Data/HOR13942_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR13942 = int(n_mRNA_intronless_HOR13942[0])/n_intronless_HOR13942
print(avg_mRNA_intronless_HOR13942)
avg_mRNA_intronpoor_HOR13942 = int(n_mRNA_intronpoor_HOR13942[0])/n_intronpoor_HOR13942
print(avg_mRNA_intronpoor_HOR13942)
avg_mRNA_intronrich_HOR13942 = int(n_mRNA_intronrich_HOR13942[0])/n_intronrich_HOR13942
print(avg_mRNA_intronrich_HOR13942)

In [ ]:
### --- Basic statistics of HOR13942 introns
introns_HOR13942['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR13942 introns length
introns_HOR13942['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR13942 introns length without outliers
introns_HOR13942.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.12.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR13942

In [ ]:
! mv Data/HOR13942_* Data/HOR13942/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR13942

In [ ]:
! mv Output/HOR13942_* Output/HOR13942/

In [ ]:
! ls Output/

## 3.13 HOR21599 - Gene projection

### 3.13.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR21599.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/23c55a21-9a5e-4edf-9ce3-ac3780213e4a/1/DOWNLOAD

### 3.13.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR21599.gff3  > Data/HOR21599_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR21599_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR21599_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR21599 = pd.read_csv('Data/HOR21599_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR21599.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR21599 = df_HOR21599.loc[df_HOR21599['type'].isin(['exon'])]
exon_HOR21599.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR21599.to_csv('Data/HOR21599_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR21599 = df_HOR21599.loc[df_HOR21599['strand'].isin(['+'])]
forw_HOR21599.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR21599 = df_HOR21599.loc[df_HOR21599['strand'].isin(['-'])]
rev_HOR21599.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR21599 = forw_HOR21599.loc[df_HOR21599['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR21599.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR21599 = rev_HOR21599.loc[df_HOR21599['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR21599.head()

No UTRs.

### 3.13.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR21599
extract_tot_introns_gp('Data/HOR21599_exon.tsv', 'Output/HOR21599_introns.tsv')

In [ ]:
introns_HOR21599 = pd.read_csv('Output/HOR21599_introns.tsv', sep = '\t')
introns_HOR21599[:100]

In [ ]:
### --- Counting introns type in HOR21599
introns_dictionary_HOR21599 = counting_introns_type_gp(exon_HOR21599)

In [ ]:
n_intronless_HOR21599, n_intronpoor_HOR21599, n_intronrich_HOR21599, intronless_HOR21599, intronpoor_HOR21599, intronrich_HOR21599 = splitting_introns_type(introns_dictionary_HOR21599)

In [ ]:
plotting_intron_types(n_intronless_HOR21599, n_intronpoor_HOR21599, n_intronrich_HOR21599, 'Genes HOR21599 Distribution')

In [ ]:
### --- Creating the list of HOR21599 gene IDs to check the number of transcripts using command line
intronless_file_HOR21599 = open('Output/HOR21599_intronlessIDs.txt', 'a')
intronpoor_file_HOR21599 = open('Output/HOR21599_intronpoorIDs.txt', 'a')
intronrich_file_HOR21599 = open('Output/HOR21599_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR21599)):
    if i == len(intronless_HOR21599) -1:
        intronless_file_HOR21599.write(intronless_HOR21599[i])
    else:
        intronless_file_HOR21599.write(intronless_HOR21599[i] + '\n')
intronless_file_HOR21599.close()

for i in range(len(intronpoor_HOR21599)):
    if i == len(intronpoor_HOR21599) -1:
        intronpoor_file_HOR21599.write(intronpoor_HOR21599[i])
    else:
        intronpoor_file_HOR21599.write(intronpoor_HOR21599[i] + '\n')
intronpoor_file_HOR21599.close()

for i in range(len(intronrich_HOR21599)):
    if i == len(intronrich_HOR21599) -1:
        intronrich_file_HOR21599.write(intronrich_HOR21599[i])
    else:
        intronrich_file_HOR21599.write(intronrich_HOR21599[i]+'\n')
intronrich_file_HOR21599.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR21599 = ! grep -f Output/HOR21599_intronlessIDs.txt Data/HOR21599_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR21599 = ! grep -f Output/HOR21599_intronpoorIDs.txt Data/HOR21599_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR21599 = ! grep -f Output/HOR21599_intronrichIDs.txt Data/HOR21599_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR21599 = int(n_mRNA_intronless_HOR21599[0])/n_intronless_HOR21599
print(avg_mRNA_intronless_HOR21599)
avg_mRNA_intronpoor_HOR21599 = int(n_mRNA_intronpoor_HOR21599[0])/n_intronpoor_HOR21599
print(avg_mRNA_intronpoor_HOR21599)
avg_mRNA_intronrich_HOR21599 = int(n_mRNA_intronrich_HOR21599[0])/n_intronrich_HOR21599
print(avg_mRNA_intronrich_HOR21599)

No alternative splicing.

In [ ]:
### --- Basic statistics of HOR21599 introns
introns_HOR21599['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR21599 introns length
introns_HOR21599['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR21599 introns length without outliers
introns_HOR21599.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.13.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR21599

In [ ]:
! mv Data/HOR21599_* Data/HOR21599/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR21599

In [ ]:
! mv Output/HOR21599_* Output/HOR21599/

In [ ]:
! ls Output/

## 3.14  HOR3081 - Gene projection

### 3.14.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR3081.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/9615c59f-4b06-44bf-ad99-e75c0afb4272/1/DOWNLOAD

### 3.14.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR3081.gff3  > Data/HOR3081_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR3081_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR3081_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR3081 = pd.read_csv('Data/HOR3081_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR3081.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR3081 = df_HOR3081.loc[df_HOR3081['type'].isin(['exon'])]
exon_HOR3081.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR3081.to_csv('Data/HOR3081_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR3081 = df_HOR3081.loc[df_HOR3081['strand'].isin(['+'])]
forw_HOR3081.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR3081 = df_HOR3081.loc[df_HOR3081['strand'].isin(['-'])]
rev_HOR3081.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR3081 = forw_HOR3081.loc[df_HOR3081['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR3081.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR3081 = rev_HOR3081.loc[df_HOR3081['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR3081.head()

No UTRs.

### 3.14.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR3081
extract_tot_introns_gp('Data/HOR3081_exon.tsv', 'Output/HOR3081_introns.tsv')

In [ ]:
introns_HOR3081 = pd.read_csv('Output/HOR3081_introns.tsv', sep = '\t')
introns_HOR3081[:100]

In [ ]:
### --- Counting introns type in HOR3081
introns_dictionary_HOR3081 = counting_introns_type_gp(exon_HOR3081)

In [ ]:
n_intronless_HOR3081, n_intronpoor_HOR3081, n_intronrich_HOR3081, intronless_HOR3081, intronpoor_HOR3081, intronrich_HOR3081 = splitting_introns_type(introns_dictionary_HOR3081)

In [ ]:
plotting_intron_types(n_intronless_HOR3081, n_intronpoor_HOR3081, n_intronrich_HOR3081, 'Genes HOR3081 Distribution')

In [ ]:
### --- Creating the list of HOR3081 gene IDs to check the number of transcripts using command line
intronless_file_HOR3081 = open('Output/HOR3081_intronlessIDs.txt', 'a')
intronpoor_file_HOR3081 = open('Output/HOR3081_intronpoorIDs.txt', 'a')
intronrich_file_HOR3081 = open('Output/HOR3081_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR3081)):
    if i == len(intronless_HOR3081) -1:
        intronless_file_HOR3081.write(intronless_HOR3081[i])
    else:
        intronless_file_HOR3081.write(intronless_HOR3081[i] + '\n')
intronless_file_HOR3081.close()

for i in range(len(intronpoor_HOR3081)):
    if i == len(intronpoor_HOR3081) -1:
        intronpoor_file_HOR3081.write(intronpoor_HOR3081[i])
    else:
        intronpoor_file_HOR3081.write(intronpoor_HOR3081[i] + '\n')
intronpoor_file_HOR3081.close()

for i in range(len(intronrich_HOR3081)):
    if i == len(intronrich_HOR3081) -1:
        intronrich_file_HOR3081.write(intronrich_HOR3081[i])
    else:
        intronrich_file_HOR3081.write(intronrich_HOR3081[i]+'\n')
intronrich_file_HOR3081.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR3081 = ! grep -f Output/HOR3081_intronlessIDs.txt Data/HOR3081_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR3081 = ! grep -f Output/HOR3081_intronpoorIDs.txt Data/HOR3081_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR3081 = ! grep -f Output/HOR3081_intronrichIDs.txt Data/HOR3081_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR3081 = int(n_mRNA_intronless_HOR3081[0])/n_intronless_HOR3081
print(avg_mRNA_intronless_HOR3081)
avg_mRNA_intronpoor_HOR3081 = int(n_mRNA_intronpoor_HOR3081[0])/n_intronpoor_HOR3081
print(avg_mRNA_intronpoor_HOR3081)
avg_mRNA_intronrich_HOR3081 = int(n_mRNA_intronrich_HOR3081[0])/n_intronrich_HOR3081
print(avg_mRNA_intronrich_HOR3081)

No alternative splicing. 

In [ ]:
### --- Basic statistics of HOR3081 introns
introns_HOR3081['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR3081 introns length
introns_HOR3081['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR3081 introns length without outliers
introns_HOR3081.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.14.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR3081

In [ ]:
! mv Data/HOR3081_* Data/HOR3081/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR3081 

In [ ]:
! mv Output/HOR3081_* Output/HOR3081/

In [ ]:
! ls Output/

## 3.15 HOR3365 - Gene projection

### 3.15.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR3365.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/7c5ccbfb-0a17-4621-8d07-0fb1b16165d0/1/DOWNLOAD

### 3.15.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR3365.gff3  > Data/HOR3365_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR3365_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR3365_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR3365 = pd.read_csv('Data/HOR3365_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR3365.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR3365 = df_HOR3365.loc[df_HOR3365['type'].isin(['exon'])]
exon_HOR3365.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR3365.to_csv('Data/HOR3365_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR3365 = df_HOR3365.loc[df_HOR3365['strand'].isin(['+'])]
forw_HOR3365.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR3365 = df_HOR3365.loc[df_HOR3365['strand'].isin(['-'])]
rev_HOR3365.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR3365 = forw_HOR3365.loc[df_HOR3365['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR3365.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR3365 = rev_HOR3365.loc[df_HOR3365['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR3365.head()

No UTRs.

### 3.15.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR3365
extract_tot_introns_gp('Data/HOR3365_exon.tsv', 'Output/HOR3365_introns.tsv')

In [ ]:
introns_HOR3365 = pd.read_csv('Output/HOR3365_introns.tsv', sep = '\t')
introns_HOR3365[:100]

In [ ]:
### --- Counting introns type in HOR3365
introns_dictionary_HOR3365 = counting_introns_type_gp(exon_HOR3365)

In [ ]:
n_intronless_HOR3365, n_intronpoor_HOR3365, n_intronrich_HOR3365, intronless_HOR3365, intronpoor_HOR3365, intronrich_HOR3365 = splitting_introns_type(introns_dictionary_HOR3365)

In [ ]:
plotting_intron_types(n_intronless_HOR3365, n_intronpoor_HOR3365, n_intronrich_HOR3365, 'Genes HOR3365 Distribution')

In [ ]:
### --- Creating the list of HOR3365 gene IDs to check the number of transcripts using command line
intronless_file_HOR3365 = open('Output/HOR3365_intronlessIDs.txt', 'a')
intronpoor_file_HOR3365 = open('Output/HOR3365_intronpoorIDs.txt', 'a')
intronrich_file_HOR3365 = open('Output/HOR3365_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR3365)):
    if i == len(intronless_HOR3365) -1:
        intronless_file_HOR3365.write(intronless_HOR3365[i])
    else:
        intronless_file_HOR3365.write(intronless_HOR3365[i] + '\n')
intronless_file_HOR3365.close()

for i in range(len(intronpoor_HOR3365)):
    if i == len(intronpoor_HOR3365) -1:
        intronpoor_file_HOR3365.write(intronpoor_HOR3365[i])
    else:
        intronpoor_file_HOR3365.write(intronpoor_HOR3365[i] + '\n')
intronpoor_file_HOR3365.close()

for i in range(len(intronrich_HOR3365)):
    if i == len(intronrich_HOR3365) -1:
        intronrich_file_HOR3365.write(intronrich_HOR3365[i])
    else:
        intronrich_file_HOR3365.write(intronrich_HOR3365[i]+'\n')
intronrich_file_HOR3365.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR3365 = ! grep -f Output/HOR3365_intronlessIDs.txt Data/HOR3365_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR3365 = ! grep -f Output/HOR3365_intronpoorIDs.txt Data/HOR3365_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR3365 = ! grep -f Output/HOR3365_intronrichIDs.txt Data/HOR3365_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR3365 = int(n_mRNA_intronless_HOR3365[0])/n_intronless_HOR3365
print(avg_mRNA_intronless_HOR3365)
avg_mRNA_intronpoor_HOR3365 = int(n_mRNA_intronpoor_HOR3365[0])/n_intronpoor_HOR3365
print(avg_mRNA_intronpoor_HOR3365)
avg_mRNA_intronrich_HOR3365 = int(n_mRNA_intronrich_HOR3365[0])/n_intronrich_HOR3365
print(avg_mRNA_intronrich_HOR3365)

No alternative splicing.

In [ ]:
### --- Basic statistics of HOR3365 introns
introns_HOR3365['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR3365 introns length
introns_HOR3365['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR3365 introns length without outliers
introns_HOR3365.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.15.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR3365

In [ ]:
! mv Data/HOR3365_* Data/HOR3365

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR3365

In [ ]:
! mv Output/HOR3365_* Output/HOR3365/

In [ ]:
! ls Output/

## 3.16 HOR7552 - Gene projection

### 3.16.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR7552.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/c1e60cfc-4331-4c9f-a4bb-22e236900701/1/DOWNLOAD

### 3.16.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR7552.gff3  > Data/HOR7552_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR7552_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR7552_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR7552 = pd.read_csv('Data/HOR7552_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR7552.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR7552 = df_HOR7552.loc[df_HOR7552['type'].isin(['exon'])]
exon_HOR7552.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR7552.to_csv('Data/HOR7552_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR7552 = df_HOR7552.loc[df_HOR7552['strand'].isin(['+'])]
forw_HOR7552.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR7552 = df_HOR7552.loc[df_HOR7552['strand'].isin(['-'])]
rev_HOR7552.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR7552 = forw_HOR7552.loc[df_HOR7552['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR7552.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR7552 = rev_HOR7552.loc[df_HOR7552['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR7552.head()

No UTRs.

### 3.16.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR7552
extract_tot_introns_gp('Data/HOR7552_exon.tsv', 'Output/HOR7552_introns.tsv')

In [ ]:
introns_HOR7552 = pd.read_csv('Output/HOR7552_introns.tsv', sep = '\t')
introns_HOR7552[:100]

In [ ]:
### --- Counting introns type in HOR7552
introns_dictionary_HOR7552 = counting_introns_type_gp(exon_HOR7552)

In [ ]:
n_intronless_HOR7552, n_intronpoor_HOR7552, n_intronrich_HOR7552, intronless_HOR7552, intronpoor_HOR7552, intronrich_HOR7552 = splitting_introns_type(introns_dictionary_HOR7552)

In [ ]:
plotting_intron_types(n_intronless_HOR7552, n_intronpoor_HOR7552, n_intronrich_HOR7552, 'Genes HOR7552 Distribution')

In [ ]:
### --- Creating the list of HOR7552 gene IDs to check the number of transcripts using command line
intronless_file_HOR7552 = open('Output/HOR7552_intronlessIDs.txt', 'a')
intronpoor_file_HOR7552 = open('Output/HOR7552_intronpoorIDs.txt', 'a')
intronrich_file_HOR7552 = open('Output/HOR7552_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR7552)):
    if i == len(intronless_HOR7552) -1:
        intronless_file_HOR7552.write(intronless_HOR7552[i])
    else:
        intronless_file_HOR7552.write(intronless_HOR7552[i] + '\n')
intronless_file_HOR7552.close()

for i in range(len(intronpoor_HOR7552)):
    if i == len(intronpoor_HOR7552) -1:
        intronpoor_file_HOR7552.write(intronpoor_HOR7552[i])
    else:
        intronpoor_file_HOR7552.write(intronpoor_HOR7552[i] + '\n')
intronpoor_file_HOR7552.close()

for i in range(len(intronrich_HOR7552)):
    if i == len(intronrich_HOR7552) -1:
        intronrich_file_HOR7552.write(intronrich_HOR7552[i])
    else:
        intronrich_file_HOR7552.write(intronrich_HOR7552[i]+'\n')
intronrich_file_HOR7552.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR7552 = ! grep -f Output/HOR7552_intronlessIDs.txt Data/HOR7552_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR7552 = ! grep -f Output/HOR7552_intronpoorIDs.txt Data/HOR7552_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR7552 = ! grep -f Output/HOR7552_intronrichIDs.txt Data/HOR7552_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR7552 = int(n_mRNA_intronless_HOR7552[0])/n_intronless_HOR7552
print(avg_mRNA_intronless_HOR7552)
avg_mRNA_intronpoor_HOR7552 = int(n_mRNA_intronpoor_HOR7552[0])/n_intronpoor_HOR7552
print(avg_mRNA_intronpoor_HOR7552)
avg_mRNA_intronrich_HOR7552 = int(n_mRNA_intronrich_HOR7552[0])/n_intronrich_HOR7552
print(avg_mRNA_intronrich_HOR7552)

No alternative splicing. 

In [ ]:
### --- Basic statistics of HOR7552 introns
introns_HOR7552['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR7552 introns length
introns_HOR7552['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR7552 introns length without outliers
introns_HOR7552.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.16.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR7552

In [ ]:
! mv Data/HOR7552_* Data/HOR7552/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR7552

In [ ]:
! mv Output/HOR7552_* Output/HOR7552/

In [ ]:
! ls Output/

## 3.17 HOR8148 - Gene projection

### 3.17.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR8148.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/178616bc-965a-4c95-b375-8dc0fee67429/1/DOWNLOAD

### 3.17.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR8148.gff3  > Data/HOR8148_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR8148_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR8148_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR8148 = pd.read_csv('Data/HOR8148_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR8148.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR8148 = df_HOR8148.loc[df_HOR8148['type'].isin(['exon'])]
exon_HOR8148.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR8148.to_csv('Data/HOR8148_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR8148 = df_HOR8148.loc[df_HOR8148['strand'].isin(['+'])]
forw_HOR8148.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR8148 = df_HOR8148.loc[df_HOR8148['strand'].isin(['-'])]
rev_HOR8148.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR8148 = forw_HOR8148.loc[df_HOR8148['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR8148.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR8148 = rev_HOR8148.loc[df_HOR8148['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR8148.head()

No UTRs.

### 3.17.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from HOR8148
extract_tot_introns_gp('Data/HOR8148_exon.tsv', 'Output/HOR8148_introns.tsv')

In [ ]:
introns_HOR8148 = pd.read_csv('Output/HOR8148_introns.tsv', sep = '\t')
introns_HOR8148[:100]

In [ ]:
### --- Counting introns type in HOR8148
introns_dictionary_HOR8148 = counting_introns_type_gp(exon_HOR8148)

In [ ]:
n_intronless_HOR8148, n_intronpoor_HOR8148, n_intronrich_HOR8148, intronless_HOR8148, intronpoor_HOR8148, intronrich_HOR8148 = splitting_introns_type(introns_dictionary_HOR8148)

In [ ]:
plotting_intron_types(n_intronless_HOR8148, n_intronpoor_HOR8148, n_intronrich_HOR8148, 'Genes HOR8148 Distribution')

In [ ]:
### --- Creating the list of HOR8148 gene IDs to check the number of transcripts using command line
intronless_file_HOR8148 = open('Output/HOR8148_intronlessIDs.txt', 'a')
intronpoor_file_HOR8148 = open('Output/HOR8148_intronpoorIDs.txt', 'a')
intronrich_file_HOR8148 = open('Output/HOR8148_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR8148)):
    if i == len(intronless_HOR8148) -1:
        intronless_file_HOR8148.write(intronless_HOR8148[i])
    else:
        intronless_file_HOR8148.write(intronless_HOR8148[i] + '\n')
intronless_file_HOR8148.close()

for i in range(len(intronpoor_HOR8148)):
    if i == len(intronpoor_HOR8148) -1:
        intronpoor_file_HOR8148.write(intronpoor_HOR8148[i])
    else:
        intronpoor_file_HOR8148.write(intronpoor_HOR8148[i] + '\n')
intronpoor_file_HOR8148.close()

for i in range(len(intronrich_HOR8148)):
    if i == len(intronrich_HOR8148) -1:
        intronrich_file_HOR8148.write(intronrich_HOR8148[i])
    else:
        intronrich_file_HOR8148.write(intronrich_HOR8148[i]+'\n')
intronrich_file_HOR8148.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR8148 = ! grep -f Output/HOR8148_intronlessIDs.txt Data/HOR8148_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR8148 = ! grep -f Output/HOR8148_intronpoorIDs.txt Data/HOR8148_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR8148 = ! grep -f Output/HOR8148_intronrichIDs.txt Data/HOR8148_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR8148 = int(n_mRNA_intronless_HOR8148[0])/n_intronless_HOR8148
print(avg_mRNA_intronless_HOR8148)
avg_mRNA_intronpoor_HOR8148 = int(n_mRNA_intronpoor_HOR8148[0])/n_intronpoor_HOR8148
print(avg_mRNA_intronpoor_HOR8148)
avg_mRNA_intronrich_HOR8148 = int(n_mRNA_intronrich_HOR8148[0])/n_intronrich_HOR8148
print(avg_mRNA_intronrich_HOR8148)

No alternative splicing.

In [ ]:
### --- Basic statistics of HOR8148 introns
introns_HOR8148['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR8148 introns length
introns_HOR8148['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR8148 introns length without outliers
introns_HOR8148.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.17.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR8148

In [ ]:
! mv Data/HOR8148_* Data/HOR8148/

In [ ]:
! ls Data

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR8148

In [ ]:
! mv Output/HOR8148_* Output/HOR8148/

In [ ]:
! ls Output/

## 3.18 HOR9043 - Gene projection

### 3.18.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/HOR9043.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/8f5a2569-f234-4001-97df-66bcd54c4b9d/1/DOWNLOAD

### 3.18.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/HOR9043.gff3  > Data/HOR9043_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/HOR9043_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/HOR9043_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_HOR9043 = pd.read_csv('Data/HOR9043_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_HOR9043.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_HOR9043 = df_HOR9043.loc[df_HOR9043['type'].isin(['exon'])]
exon_HOR9043.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_HOR9043.to_csv('Data/HOR9043_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_HOR9043 = df_HOR9043.loc[df_HOR9043['strand'].isin(['+'])]
forw_HOR9043.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_HOR9043 = df_HOR9043.loc[df_HOR9043['strand'].isin(['-'])]
rev_HOR9043.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_HOR9043 = forw_HOR9043.loc[df_HOR9043['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_HOR9043.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_HOR9043 = rev_HOR9043.loc[df_HOR9043['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_HOR9043.head()

No UTRs.

### 3.18.3 Extracting introns from the whole genome 

In [ ]:
### --- Extracting introns from HOR9043
extract_tot_introns_gp('Data/HOR9043_exon.tsv', 'Output/HOR9043_introns.tsv')

In [ ]:
introns_HOR9043 = pd.read_csv('Output/HOR9043_introns.tsv', sep = '\t')
introns_HOR9043[:100]

In [ ]:
### --- Counting introns type in HOR9043
introns_dictionary_HOR9043 = counting_introns_type_gp(exon_HOR9043)

In [ ]:
n_intronless_HOR9043, n_intronpoor_HOR9043, n_intronrich_HOR9043, intronless_HOR9043, intronpoor_HOR9043, intronrich_HOR9043 = splitting_introns_type(introns_dictionary_HOR9043)

In [ ]:
plotting_intron_types(n_intronless_HOR9043, n_intronpoor_HOR9043, n_intronrich_HOR9043, 'Genes HOR9043 Distribution')

In [ ]:
### --- Creating the list of HOR9043 gene IDs to check the number of transcripts using command line
intronless_file_HOR9043 = open('Output/HOR9043_intronlessIDs.txt', 'a')
intronpoor_file_HOR9043 = open('Output/HOR9043_intronpoorIDs.txt', 'a')
intronrich_file_HOR9043 = open('Output/HOR9043_intronrichIDs.txt', 'a')
for i in range(len(intronless_HOR9043)):
    if i == len(intronless_HOR9043) -1:
        intronless_file_HOR9043.write(intronless_HOR9043[i])
    else:
        intronless_file_HOR9043.write(intronless_HOR9043[i] + '\n')
intronless_file_HOR9043.close()

for i in range(len(intronpoor_HOR9043)):
    if i == len(intronpoor_HOR9043) -1:
        intronpoor_file_HOR9043.write(intronpoor_HOR9043[i])
    else:
        intronpoor_file_HOR9043.write(intronpoor_HOR9043[i] + '\n')
intronpoor_file_HOR9043.close()

for i in range(len(intronrich_HOR9043)):
    if i == len(intronrich_HOR9043) -1:
        intronrich_file_HOR9043.write(intronrich_HOR9043[i])
    else:
        intronrich_file_HOR9043.write(intronrich_HOR9043[i]+'\n')
intronrich_file_HOR9043.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_HOR9043 = ! grep -f Output/HOR9043_intronlessIDs.txt Data/HOR9043_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_HOR9043 = ! grep -f Output/HOR9043_intronpoorIDs.txt Data/HOR9043_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_HOR9043 = ! grep -f Output/HOR9043_intronrichIDs.txt Data/HOR9043_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_HOR9043 = int(n_mRNA_intronless_HOR9043[0])/n_intronless_HOR9043
print(avg_mRNA_intronless_HOR9043)
avg_mRNA_intronpoor_HOR9043 = int(n_mRNA_intronpoor_HOR9043[0])/n_intronpoor_HOR9043
print(avg_mRNA_intronpoor_HOR9043)
avg_mRNA_intronrich_HOR9043 = int(n_mRNA_intronrich_HOR9043[0])/n_intronrich_HOR9043
print(avg_mRNA_intronrich_HOR9043)

In [ ]:
### --- Basic statistics of HOR9043 introns
introns_HOR9043['length'].describe()

In [ ]:
### --- Plotting the distribution of HOR9043 introns length
introns_HOR9043['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of HOR9043 introns length without outliers
introns_HOR9043.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.18.4 Re-organizing files and directories 

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/HOR9043

In [ ]:
! mv Data/HOR9043_* Data/HOR9043/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/HOR9043

In [ ]:
! mv Output/HOR9043_* Output/HOR9043/

In [ ]:
! ls Output/

## 3.19 Igri - Gene projection

### 3.19.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Igri.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/053b94a0-5d6e-4339-b2d5-3de486760c32/1/DOWNLOAD

### 3.19.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Igri.gff3  > Data/Igri_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Igri_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Igri_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Igri = pd.read_csv('Data/Igri_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Igri.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Igri = df_Igri.loc[df_Igri['type'].isin(['exon'])]
exon_Igri.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Igri.to_csv('Data/Igri_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Igri = df_Igri.loc[df_Igri['strand'].isin(['+'])]
forw_Igri.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Igri = df_Igri.loc[df_Igri['strand'].isin(['-'])]
rev_Igri.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Igri = forw_Igri.loc[df_Igri['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Igri.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Igri = rev_Igri.loc[df_Igri['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Igri.head()

No UTRs.

### 3.19.3 Extracting introns from the whole genome 

In [ ]:
### --- Extracting introns from Igri
extract_tot_introns_gp('Data/Igri_exon.tsv', 'Output/Igri_introns.tsv')

In [ ]:
introns_Igri = pd.read_csv('Output/Igri_introns.tsv', sep = '\t')
introns_Igri[:100]

In [ ]:
### --- Counting introns type in Igri
introns_dictionary_Igri = counting_introns_type_gp(exon_Igri)

In [ ]:
n_intronless_Igri, n_intronpoor_Igri, n_intronrich_Igri, intronless_Igri, intronpoor_Igri, intronrich_Igri = splitting_introns_type(introns_dictionary_Igri)

In [ ]:
plotting_intron_types(n_intronless_Igri, n_intronpoor_Igri, n_intronrich_Igri, 'Genes Igri Distribution')

In [ ]:
### --- Creating the list of Igri gene IDs to check the number of transcripts using command line
intronless_file_Igri = open('Output/Igri_intronlessIDs.txt', 'a')
intronpoor_file_Igri = open('Output/Igri_intronpoorIDs.txt', 'a')
intronrich_file_Igri = open('Output/Igri_intronrichIDs.txt', 'a')
for i in range(len(intronless_Igri)):
    if i == len(intronless_Igri) -1:
        intronless_file_Igri.write(intronless_Igri[i])
    else:
        intronless_file_Igri.write(intronless_Igri[i] + '\n')
intronless_file_Igri.close()

for i in range(len(intronpoor_Igri)):
    if i == len(intronpoor_Igri) -1:
        intronpoor_file_Igri.write(intronpoor_Igri[i])
    else:
        intronpoor_file_Igri.write(intronpoor_Igri[i] + '\n')
intronpoor_file_Igri.close()

for i in range(len(intronrich_Igri)):
    if i == len(intronrich_Igri) -1:
        intronrich_file_Igri.write(intronrich_Igri[i])
    else:
        intronrich_file_Igri.write(intronrich_Igri[i]+'\n')
intronrich_file_Igri.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Igri = ! grep -f Output/Igri_intronlessIDs.txt Data/Igri_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Igri = ! grep -f Output/Igri_intronpoorIDs.txt Data/Igri_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Igri = ! grep -f Output/Igri_intronrichIDs.txt Data/Igri_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Igri = int(n_mRNA_intronless_Igri[0])/n_intronless_Igri
print(avg_mRNA_intronless_Igri)
avg_mRNA_intronpoor_Igri = int(n_mRNA_intronpoor_Igri[0])/n_intronpoor_Igri
print(avg_mRNA_intronpoor_Igri)
avg_mRNA_intronrich_Igri = int(n_mRNA_intronrich_Igri[0])/n_intronrich_Igri
print(avg_mRNA_intronrich_Igri)

In [ ]:
### --- Basic statistics of Igri introns
introns_Igri['length'].describe()

In [ ]:
### --- Plotting the distribution of Igri introns length
introns_Igri['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Igri introns length without outliers
introns_Igri.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.19.4 Re-organizing files and directories 

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Igri 

In [ ]:
! mv Data/Igri_* Data/Igri/

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Igri

In [ ]:
! mv Output/Igri_* Output/Igri/

In [ ]:
! ls Output/

## 3.20 Morex - Gene projection

### 3.20.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/Morex_gp.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/d8559e36-1879-406f-88de-f55786ce39eb/1/DOWNLOAD

### 3.20.2 Processing Data

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/Morex_gp.gff3  > Data/Morex_gp_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/Morex_gp_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/Morex_gp_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_Morex_gp = pd.read_csv('Data/Morex_gp_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_Morex_gp.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_Morex_gp = df_Morex_gp.loc[df_Morex_gp['type'].isin(['exon'])]
exon_Morex_gp.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_Morex_gp.to_csv('Data/Morex_gp_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_Morex_gp = df_Morex_gp.loc[df_Morex_gp['strand'].isin(['+'])]
forw_Morex_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_Morex_gp = df_Morex_gp.loc[df_Morex_gp['strand'].isin(['-'])]
rev_Morex_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_Morex_gp = forw_Morex_gp.loc[df_Morex_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_Morex_gp.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_Morex_gp = rev_Morex_gp.loc[df_Morex_gp['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_Morex_gp.head()

No UTRs.

### 3.20.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from Morex_gp
extract_tot_introns_gp('Data/Morex_gp_exon.tsv', 'Output/Morex_gp_introns.tsv')

In [ ]:
introns_Morex_gp = pd.read_csv('Output/Morex_gp_introns.tsv', sep = '\t')
introns_Morex_gp[:100]

In [ ]:
### --- Counting introns type in Morex_gp
introns_dictionary_Morex_gp = counting_introns_type_gp(exon_Morex_gp)

In [ ]:
n_intronless_Morex_gp, n_intronpoor_Morex_gp, n_intronrich_Morex_gp, intronless_Morex_gp, intronpoor_Morex_gp, intronrich_Morex_gp = splitting_introns_type(introns_dictionary_Morex_gp)

In [ ]:
plotting_intron_types(n_intronless_Morex_gp, n_intronpoor_Morex_gp, n_intronrich_Morex_gp, 'Genes Morex_gp Distribution')

In [ ]:
### --- Creating the list of Morex_gp gene IDs to check the number of transcripts using command line
intronless_file_Morex_gp = open('Output/Morex_gp_intronlessIDs.txt', 'a')
intronpoor_file_Morex_gp = open('Output/Morex_gp_intronpoorIDs.txt', 'a')
intronrich_file_Morex_gp = open('Output/Morex_gp_intronrichIDs.txt', 'a')
for i in range(len(intronless_Morex_gp)):
    if i == len(intronless_Morex_gp) -1:
        intronless_file_Morex_gp.write(intronless_Morex_gp[i])
    else:
        intronless_file_Morex_gp.write(intronless_Morex_gp[i] + '\n')
intronless_file_Morex_gp.close()

for i in range(len(intronpoor_Morex_gp)):
    if i == len(intronpoor_Morex_gp) -1:
        intronpoor_file_Morex_gp.write(intronpoor_Morex_gp[i])
    else:
        intronpoor_file_Morex_gp.write(intronpoor_Morex_gp[i] + '\n')
intronpoor_file_Morex_gp.close()

for i in range(len(intronrich_Morex_gp)):
    if i == len(intronrich_Morex_gp) -1:
        intronrich_file_Morex_gp.write(intronrich_Morex_gp[i])
    else:
        intronrich_file_Morex_gp.write(intronrich_Morex_gp[i]+'\n')
intronrich_file_Morex_gp.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_Morex_gp = ! grep -f Output/Morex_gp_intronlessIDs.txt Data/Morex_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_Morex_gp = ! grep -f Output/Morex_gp_intronpoorIDs.txt Data/Morex_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_Morex_gp = ! grep -f Output/Morex_gp_intronrichIDs.txt Data/Morex_gp_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_Morex_gp = int(n_mRNA_intronless_Morex_gp[0])/n_intronless_Morex_gp
print(avg_mRNA_intronless_Morex_gp)
avg_mRNA_intronpoor_Morex_gp = int(n_mRNA_intronpoor_Morex_gp[0])/n_intronpoor_Morex_gp
print(avg_mRNA_intronpoor_Morex_gp)
avg_mRNA_intronrich_Morex_gp = int(n_mRNA_intronrich_Morex_gp[0])/n_intronrich_Morex_gp
print(avg_mRNA_intronrich_Morex_gp)

No alternative splicing.

In [ ]:
### --- Basic statistics of Morex_gp introns
introns_Morex_gp['length'].describe()

In [ ]:
### --- Plotting the distribution of Morex_gp introns length
introns_Morex_gp['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of Morex_gp introns length without outliers
introns_Morex_gp.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.20.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/Morex_gp

In [ ]:
! mv Data/Morex_gp_* Data/Morex_gp

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/Morex_gp

In [ ]:
! mv Output/Morex_gp_* Output/Morex_gp

In [ ]:
! ls Output/

## 3.21 OUN333 - Gene projection

### 3.21.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/OUN333.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/9b5ea42c-6b27-4bd7-a66e-3210ded3ece5/1/DOWNLOAD

### 3.21.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/OUN333.gff3  > Data/OUN333_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/OUN333_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/OUN333_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_OUN333 = pd.read_csv('Data/OUN333_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_OUN333.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_OUN333 = df_OUN333.loc[df_OUN333['type'].isin(['exon'])]
exon_OUN333.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_OUN333.to_csv('Data/OUN333_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_OUN333 = df_OUN333.loc[df_OUN333['strand'].isin(['+'])]
forw_OUN333.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_OUN333 = df_OUN333.loc[df_OUN333['strand'].isin(['-'])]
rev_OUN333.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_OUN333 = forw_OUN333.loc[df_OUN333['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_OUN333.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_OUN333 = rev_OUN333.loc[df_OUN333['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_OUN333.head()

No UTRs.

### 3.21.3 Extracting introns from the whole genome 

In [ ]:
### --- Extracting introns from OUN333
extract_tot_introns_gp('Data/OUN333_exon.tsv', 'Output/OUN333_introns.tsv')

In [ ]:
introns_OUN333 = pd.read_csv('Output/OUN333_introns.tsv', sep = '\t')
introns_OUN333[:100]

In [ ]:
### --- Counting introns type in OUN333
introns_dictionary_OUN333 = counting_introns_type_gp(exon_OUN333)

In [ ]:
n_intronless_OUN333, n_intronpoor_OUN333, n_intronrich_OUN333, intronless_OUN333, intronpoor_OUN333, intronrich_OUN333 = splitting_introns_type(introns_dictionary_OUN333)

In [ ]:
plotting_intron_types(n_intronless_OUN333, n_intronpoor_OUN333, n_intronrich_OUN333, 'Genes OUN333 Distribution')

In [ ]:
### --- Creating the list of OUN333 gene IDs to check the number of transcripts using command line
intronless_file_OUN333 = open('Output/OUN333_intronlessIDs.txt', 'a')
intronpoor_file_OUN333 = open('Output/OUN333_intronpoorIDs.txt', 'a')
intronrich_file_OUN333 = open('Output/OUN333_intronrichIDs.txt', 'a')
for i in range(len(intronless_OUN333)):
    if i == len(intronless_OUN333) -1:
        intronless_file_OUN333.write(intronless_OUN333[i])
    else:
        intronless_file_OUN333.write(intronless_OUN333[i] + '\n')
intronless_file_OUN333.close()

for i in range(len(intronpoor_OUN333)):
    if i == len(intronpoor_OUN333) -1:
        intronpoor_file_OUN333.write(intronpoor_OUN333[i])
    else:
        intronpoor_file_OUN333.write(intronpoor_OUN333[i] + '\n')
intronpoor_file_OUN333.close()

for i in range(len(intronrich_OUN333)):
    if i == len(intronrich_OUN333) -1:
        intronrich_file_OUN333.write(intronrich_OUN333[i])
    else:
        intronrich_file_OUN333.write(intronrich_OUN333[i]+'\n')
intronrich_file_OUN333.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_OUN333 = ! grep -f Output/OUN333_intronlessIDs.txt Data/OUN333_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_OUN333 = ! grep -f Output/OUN333_intronpoorIDs.txt Data/OUN333_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_OUN333 = ! grep -f Output/OUN333_intronrichIDs.txt Data/OUN333_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_OUN333 = int(n_mRNA_intronless_OUN333[0])/n_intronless_OUN333
print(avg_mRNA_intronless_OUN333)
avg_mRNA_intronpoor_OUN333 = int(n_mRNA_intronpoor_OUN333[0])/n_intronpoor_OUN333
print(avg_mRNA_intronpoor_OUN333)
avg_mRNA_intronrich_OUN333 = int(n_mRNA_intronrich_OUN333[0])/n_intronrich_OUN333
print(avg_mRNA_intronrich_OUN333)

No alternative splicing.

In [ ]:
### --- Basic statistics of OUN333 introns
introns_OUN333['length'].describe()

In [ ]:
### --- Plotting the distribution of OUN333 introns length
introns_OUN333['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of OUN333 introns length without outliers
introns_OUN333.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.21.4 Re-organizing files and directories 

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/OUN333

In [ ]:
! mv Data/OUN333_* Data/OUN333

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/OUN333

In [ ]:
! mv Output/OUN333_* Output/OUN333

In [ ]:
! ls Output/

## 3.22 RGT_Planet - Gene projection

### 3.22.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/RGT_Planet.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/92f038bd-4112-42a0-aa90-431940f82159/1/DOWNLOAD

### 3.22.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/RGT_Planet.gff3  > Data/RGT_Planet_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/RGT_Planet_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/RGT_Planet_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_RGT_Planet = pd.read_csv('Data/RGT_Planet_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_RGT_Planet.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_RGT_Planet = df_RGT_Planet.loc[df_RGT_Planet['type'].isin(['exon'])]
exon_RGT_Planet.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_RGT_Planet.to_csv('Data/RGT_Planet_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_RGT_Planet = df_RGT_Planet.loc[df_RGT_Planet['strand'].isin(['+'])]
forw_RGT_Planet.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_RGT_Planet = df_RGT_Planet.loc[df_RGT_Planet['strand'].isin(['-'])]
rev_RGT_Planet.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_RGT_Planet = forw_RGT_Planet.loc[df_RGT_Planet['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_RGT_Planet.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_RGT_Planet = rev_RGT_Planet.loc[df_RGT_Planet['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_RGT_Planet.head()

No UTRs.

### 3.22.3 Extracting introns from the whole genome 

In [ ]:
### --- Extracting introns from RGT_Planet
extract_tot_introns_gp('Data/RGT_Planet_exon.tsv', 'Output/RGT_Planet_introns.tsv')

In [ ]:
introns_RGT_Planet = pd.read_csv('Output/RGT_Planet_introns.tsv', sep = '\t')
introns_RGT_Planet[:100]

In [ ]:
### --- Counting introns type in RGT_Planet
introns_dictionary_RGT_Planet = counting_introns_type_gp(exon_RGT_Planet)

In [ ]:
n_intronless_RGT_Planet, n_intronpoor_RGT_Planet, n_intronrich_RGT_Planet, intronless_RGT_Planet, intronpoor_RGT_Planet, intronrich_RGT_Planet = splitting_introns_type(introns_dictionary_RGT_Planet)

In [ ]:
plotting_intron_types(n_intronless_RGT_Planet, n_intronpoor_RGT_Planet, n_intronrich_RGT_Planet, 'Genes RGT_Planet Distribution')

In [ ]:
### --- Creating the list of RGT_Planet gene IDs to check the number of transcripts using command line
intronless_file_RGT_Planet = open('Output/RGT_Planet_intronlessIDs.txt', 'a')
intronpoor_file_RGT_Planet = open('Output/RGT_Planet_intronpoorIDs.txt', 'a')
intronrich_file_RGT_Planet = open('Output/RGT_Planet_intronrichIDs.txt', 'a')
for i in range(len(intronless_RGT_Planet)):
    if i == len(intronless_RGT_Planet) -1:
        intronless_file_RGT_Planet.write(intronless_RGT_Planet[i])
    else:
        intronless_file_RGT_Planet.write(intronless_RGT_Planet[i] + '\n')
intronless_file_RGT_Planet.close()

for i in range(len(intronpoor_RGT_Planet)):
    if i == len(intronpoor_RGT_Planet) -1:
        intronpoor_file_RGT_Planet.write(intronpoor_RGT_Planet[i])
    else:
        intronpoor_file_RGT_Planet.write(intronpoor_RGT_Planet[i] + '\n')
intronpoor_file_RGT_Planet.close()

for i in range(len(intronrich_RGT_Planet)):
    if i == len(intronrich_RGT_Planet) -1:
        intronrich_file_RGT_Planet.write(intronrich_RGT_Planet[i])
    else:
        intronrich_file_RGT_Planet.write(intronrich_RGT_Planet[i]+'\n')
intronrich_file_RGT_Planet.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_RGT_Planet = ! grep -f Output/RGT_Planet_intronlessIDs.txt Data/RGT_Planet_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_RGT_Planet = ! grep -f Output/RGT_Planet_intronpoorIDs.txt Data/RGT_Planet_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_RGT_Planet = ! grep -f Output/RGT_Planet_intronrichIDs.txt Data/RGT_Planet_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_RGT_Planet = int(n_mRNA_intronless_RGT_Planet[0])/n_intronless_RGT_Planet
print(avg_mRNA_intronless_RGT_Planet)
avg_mRNA_intronpoor_RGT_Planet = int(n_mRNA_intronpoor_RGT_Planet[0])/n_intronpoor_RGT_Planet
print(avg_mRNA_intronpoor_RGT_Planet)
avg_mRNA_intronrich_RGT_Planet = int(n_mRNA_intronrich_RGT_Planet[0])/n_intronrich_RGT_Planet
print(avg_mRNA_intronrich_RGT_Planet)

No alternative splicing. 

In [ ]:
### --- Basic statistics of RGT_Planet introns
introns_RGT_Planet['length'].describe()

In [ ]:
### --- Plotting the distribution of RGT_Planet introns length
introns_RGT_Planet['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of RGT_Planet introns length without outliers
introns_RGT_Planet.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.22.4 Re-organizing files and directories 

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/RGT_Planet

In [ ]:
! mv Data/RGT_Planet_* Data/RGT_Planet

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/RGT_Planet

In [ ]:
! mv Output/RGT_Planet_* Output/RGT_Planet

In [ ]:
! ls Output/

## 3.23 ZDM01467 - Gene projection

### 3.23.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/ZDM01467.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/6e7e8cf3-08d2-4f25-8dac-4eb70f2241dd/1/DOWNLOAD

### 3.23.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/ZDM01467.gff3  > Data/ZDM01467_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/ZDM01467_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/ZDM01467_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_ZDM01467 = pd.read_csv('Data/ZDM01467_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_ZDM01467.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_ZDM01467 = df_ZDM01467.loc[df_ZDM01467['type'].isin(['exon'])]
exon_ZDM01467.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_ZDM01467.to_csv('Data/ZDM01467_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_ZDM01467 = df_ZDM01467.loc[df_ZDM01467['strand'].isin(['+'])]
forw_ZDM01467.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_ZDM01467 = df_ZDM01467.loc[df_ZDM01467['strand'].isin(['-'])]
rev_ZDM01467.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_ZDM01467 = forw_ZDM01467.loc[df_ZDM01467['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_ZDM01467.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_ZDM01467 = rev_ZDM01467.loc[df_ZDM01467['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_ZDM01467.head()

No UTRs.

### 3.23.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from ZDM01467
extract_tot_introns_gp('Data/ZDM01467_exon.tsv', 'Output/ZDM01467_introns.tsv')

In [ ]:
introns_ZDM01467 = pd.read_csv('Output/ZDM01467_introns.tsv', sep = '\t')
introns_ZDM01467[:100]

In [ ]:
### --- Counting introns type in ZDM01467
introns_dictionary_ZDM01467 = counting_introns_type_gp(exon_ZDM01467)

In [ ]:
n_intronless_ZDM01467, n_intronpoor_ZDM01467, n_intronrich_ZDM01467, intronless_ZDM01467, intronpoor_ZDM01467, intronrich_ZDM01467 = splitting_introns_type(introns_dictionary_ZDM01467)

In [ ]:
plotting_intron_types(n_intronless_ZDM01467, n_intronpoor_ZDM01467, n_intronrich_ZDM01467, 'Genes ZDM01467 Distribution')

In [ ]:
### --- Creating the list of ZDM01467 gene IDs to check the number of transcripts using command line
intronless_file_ZDM01467 = open('Output/ZDM01467_intronlessIDs.txt', 'a')
intronpoor_file_ZDM01467 = open('Output/ZDM01467_intronpoorIDs.txt', 'a')
intronrich_file_ZDM01467 = open('Output/ZDM01467_intronrichIDs.txt', 'a')
for i in range(len(intronless_ZDM01467)):
    if i == len(intronless_ZDM01467) -1:
        intronless_file_ZDM01467.write(intronless_ZDM01467[i])
    else:
        intronless_file_ZDM01467.write(intronless_ZDM01467[i] + '\n')
intronless_file_ZDM01467.close()

for i in range(len(intronpoor_ZDM01467)):
    if i == len(intronpoor_ZDM01467) -1:
        intronpoor_file_ZDM01467.write(intronpoor_ZDM01467[i])
    else:
        intronpoor_file_ZDM01467.write(intronpoor_ZDM01467[i] + '\n')
intronpoor_file_ZDM01467.close()

for i in range(len(intronrich_ZDM01467)):
    if i == len(intronrich_ZDM01467) -1:
        intronrich_file_ZDM01467.write(intronrich_ZDM01467[i])
    else:
        intronrich_file_ZDM01467.write(intronrich_ZDM01467[i]+'\n')
intronrich_file_ZDM01467.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_ZDM01467 = ! grep -f Output/ZDM01467_intronlessIDs.txt Data/ZDM01467_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_ZDM01467 = ! grep -f Output/ZDM01467_intronpoorIDs.txt Data/ZDM01467_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_ZDM01467 = ! grep -f Output/ZDM01467_intronrichIDs.txt Data/ZDM01467_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_ZDM01467 = int(n_mRNA_intronless_ZDM01467[0])/n_intronless_ZDM01467
print(avg_mRNA_intronless_ZDM01467)
avg_mRNA_intronpoor_ZDM01467 = int(n_mRNA_intronpoor_ZDM01467[0])/n_intronpoor_ZDM01467
print(avg_mRNA_intronpoor_ZDM01467)
avg_mRNA_intronrich_ZDM01467 = int(n_mRNA_intronrich_ZDM01467[0])/n_intronrich_ZDM01467
print(avg_mRNA_intronrich_ZDM01467)

No alternative splicing.

In [ ]:
### --- Basic statistics of ZDM01467 introns
introns_ZDM01467['length'].describe()

In [ ]:
### --- Plotting the distribution of ZDM01467 introns length
introns_ZDM01467['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of ZDM01467 introns length without outliers
introns_ZDM01467.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.23.4 Re-organizing files and directories 

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/ZDM01467

In [ ]:
! mv Data/ZDM01467_* Data/ZDM01467

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/ZDM01467

In [ ]:
! mv Output/ZDM01467_* Output/ZDM01467

In [ ]:
! ls Output/

## 3.24 ZDM02064 - Gene projection

### 3.24.1 Data Collection

In [ ]:
### --- This will download the raw data to the subdirectory raw under the directory Data
! wget -O Data/raw/ZDM02064.gff3 https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/68d247e5-6cb3-4ba6-bcd6-89fa8e69a6c6/1/DOWNLOAD

### 3.24.2 Processing Data 

In [ ]:
### --- There is no need to import the libraries again
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

In [ ]:
### --- Removing all hashtags from the gff3 file
! sed '/#/d' Data/raw/ZDM02064.gff3  > Data/ZDM02064_nohashtag.gff3

In [ ]:
### --- Changing the gff3 file to a csv format file to import it with pandas
! cat Data/ZDM02064_nohashtag.gff3 | sed 's/,/--/g' | sed 's/;/--/g' | sed 's/\t/,/g' | awk -F '--' '{print $1}' > Data/ZDM02064_nohashtag.csv

In [ ]:
### --- Transforming the gff3 file to a pandas dataframe to better handle it
df_ZDM02064 = pd.read_csv('Data/ZDM02064_nohashtag.csv', header=None, names = ['chr', 'source', 'type','start','end','score','strand','phase','attributes'])
df_ZDM02064.head()

In [ ]:
### --- Building a separate dataframe containing all exons
exon_ZDM02064 = df_ZDM02064.loc[df_ZDM02064['type'].isin(['exon'])]
exon_ZDM02064.head()

In [ ]:
### --- Exporting the exon dataframe to a tsv file 
exon_ZDM02064.to_csv('Data/ZDM02064_exon.tsv',sep='\t',index=False,header=False)

In [ ]:
### --- Keeping in a separated dataframe the forward strands
forw_ZDM02064 = df_ZDM02064.loc[df_ZDM02064['strand'].isin(['+'])]
forw_ZDM02064.head()

In [ ]:
### --- Keeping in a separated dataframe the reverse strands
rev_ZDM02064 = df_ZDM02064.loc[df_ZDM02064['strand'].isin(['-'])]
rev_ZDM02064.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the forward strands
UTR_forw_ZDM02064 = forw_ZDM02064.loc[df_ZDM02064['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_forw_ZDM02064.head()

In [ ]:
### --- Keeping in a separated dataframe the UTR regions on the reverse strands
UTR_rev_ZDM02064 = rev_ZDM02064.loc[df_ZDM02064['type'].isin(['three_prime_UTR','five_prime_UTR'])]
UTR_rev_ZDM02064.head()

No UTRs.

### 3.24.3 Extracting introns from the whole genome

In [ ]:
### --- Extracting introns from ZDM02064
extract_tot_introns_gp('Data/ZDM02064_exon.tsv', 'Output/ZDM02064_introns.tsv')

In [ ]:
introns_ZDM02064 = pd.read_csv('Output/ZDM02064_introns.tsv', sep = '\t')
introns_ZDM02064[:100]

In [ ]:
### --- Counting introns type in ZDM02064
introns_dictionary_ZDM02064 = counting_introns_type_gp(exon_ZDM02064)

In [ ]:
n_intronless_ZDM02064, n_intronpoor_ZDM02064, n_intronrich_ZDM02064, intronless_ZDM02064, intronpoor_ZDM02064, intronrich_ZDM02064 = splitting_introns_type(introns_dictionary_ZDM02064)

In [ ]:
plotting_intron_types(n_intronless_ZDM02064, n_intronpoor_ZDM02064, n_intronrich_ZDM02064, 'Genes ZDM02064 Distribution')

In [ ]:
### --- Creating the list of ZDM02064 gene IDs to check the number of transcripts using command line
intronless_file_ZDM02064 = open('Output/ZDM02064_intronlessIDs.txt', 'a')
intronpoor_file_ZDM02064 = open('Output/ZDM02064_intronpoorIDs.txt', 'a')
intronrich_file_ZDM02064 = open('Output/ZDM02064_intronrichIDs.txt', 'a')
for i in range(len(intronless_ZDM02064)):
    if i == len(intronless_ZDM02064) -1:
        intronless_file_ZDM02064.write(intronless_ZDM02064[i])
    else:
        intronless_file_ZDM02064.write(intronless_ZDM02064[i] + '\n')
intronless_file_ZDM02064.close()

for i in range(len(intronpoor_ZDM02064)):
    if i == len(intronpoor_ZDM02064) -1:
        intronpoor_file_ZDM02064.write(intronpoor_ZDM02064[i])
    else:
        intronpoor_file_ZDM02064.write(intronpoor_ZDM02064[i] + '\n')
intronpoor_file_ZDM02064.close()

for i in range(len(intronrich_ZDM02064)):
    if i == len(intronrich_ZDM02064) -1:
        intronrich_file_ZDM02064.write(intronrich_ZDM02064[i])
    else:
        intronrich_file_ZDM02064.write(intronrich_ZDM02064[i]+'\n')
intronrich_file_ZDM02064.close()

In [ ]:
### --- Storing the number of transcripts in intron-less/poor/rich genes within a variable 
n_mRNA_intronless_ZDM02064 = ! grep -f Output/ZDM02064_intronlessIDs.txt Data/ZDM02064_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronpoor_ZDM02064 = ! grep -f Output/ZDM02064_intronpoorIDs.txt Data/ZDM02064_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l
n_mRNA_intronrich_ZDM02064 = ! grep -f Output/ZDM02064_intronrichIDs.txt Data/ZDM02064_nohashtag.gff3 | awk -F '\t' '{print $3}' | grep -w 'mRNA' | wc -l

In [ ]:
### --- Checking the average number of transcripts for each gene-type
avg_mRNA_intronless_ZDM02064 = int(n_mRNA_intronless_ZDM02064[0])/n_intronless_ZDM02064
print(avg_mRNA_intronless_ZDM02064)
avg_mRNA_intronpoor_ZDM02064 = int(n_mRNA_intronpoor_ZDM02064[0])/n_intronpoor_ZDM02064
print(avg_mRNA_intronpoor_ZDM02064)
avg_mRNA_intronrich_ZDM02064 = int(n_mRNA_intronrich_ZDM02064[0])/n_intronrich_ZDM02064
print(avg_mRNA_intronrich_ZDM02064)

No alternative splicing. 

In [ ]:
### --- Basic statistics of ZDM02064 introns
introns_ZDM02064['length'].describe()

In [ ]:
### --- Plotting the distribution of ZDM02064 introns length
introns_ZDM02064['length'].plot.density()

In [ ]:
### --- Plotting the boxplot of ZDM02064 introns length without outliers
introns_ZDM02064.boxplot(column='length', return_type='axes', showfliers=False) # showfliers = False -> discard outliers

### 3.24.4 Re-organizing files and directories

In [ ]:
! ls Data/

In [ ]:
! mkdir Data/ZDM02064

In [ ]:
! mv Data/ZDM02064_* Data/ZDM02064

In [ ]:
! ls Data/

In [ ]:
! ls Output/

In [ ]:
! mkdir Output/ZDM02064

In [ ]:
! mv Output/ZDM02064_* Output/ZDM02064

In [ ]:
! ls Output/

# 4 Extracting from fasta the corresponding introns UTR sequences in Hv_MorexHC

In [878]:
### --- Re-generating the full UTR introns dataframe
introns_UTR_forw_Hv_MorexHC, introns_UTR_rev_Hv_MorexHC = pd.read_csv('Output/Hv_Morex_longread/Hv_MorexHC_UTRforw_introns.tsv', sep='\t'), pd.read_csv('Output/Hv_Morex_longread/Hv_MorexHC_UTRrev_introns.tsv', sep = '\t')
frames_Hv_MorexHC = [introns_UTR_forw_Hv_MorexHC, introns_UTR_rev_Hv_MorexHC]
introns_UTR_Hv_MorexHC = pd.concat(frames_Hv_MorexHC).reset_index(drop=True)
introns_UTR_Hv_MorexHC

,ID,type,start,end,length
0,ID=HORVU.MOREX.r3.1HG0000070.1,intron five_prime_UTR,146750,146980,232
1,ID=HORVU.MOREX.r3.1HG0000080.1,intron five_prime_UTR,153530,153625,97
2,ID=HORVU.MOREX.r3.1HG0000620.1,intron five_prime_UTR,1614154,1615780,1628
3,ID=HORVU.MOREX.r3.1HG0000620.2,intron five_prime_UTR,1614154,1615780,1628
4,ID=HORVU.MOREX.r3.1HG0000620.3,intron five_prime_UTR,1614665,1615780,1117
...,...,...,...,...,...
5044,ID=HORVU.MOREX.r3.7HG0752050.1,intron five_prime_UTR,628813527,628813619,94
5045,ID=HORVU.MOREX.r3.7HG0752790.1,intron three_prime_UTR,631981108,631981202,96
5046,ID=HORVU.MOREX.r3.7HG0752950.1,intron five_prime_UTR,632146924,632147308,386
5047,ID=HORVU.MOREX.r3.UnG0753140.1,intron five_prime_UTR,475717,475945,230


In [879]:
### --- Downloading the fast from we will extract the sequences corresponding to the introns UTR.
### --- The next line has been commented due to the fact I previously downloaded this file. 

! wget -O Data/raw/Barley_MorexV3_pseudomolecules.fasta https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/b6e6a2e5-2746-4522-8465-019c8f56df7f/1/DOWNLOAD

--2021-09-04 12:40:44--  https://doi.ipk-gatersleben.de/DOI/b2f47dfb-47ff-4114-89ae-bad8dcc515a1/b6e6a2e5-2746-4522-8465-019c8f56df7f/1/DOWNLOAD
Resolving doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)... 194.94.136.144
Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4296032540 (4,0G) [text/plain]
Saving to: ‘Data/raw/Barley_MorexV3_pseudomolecules.fasta’

Data/raw/Barley_Mor 100%[===================>]   4,00G  1,26MB/s    in 53m 25s 

2021-09-04 13:34:10 (1,28 MB/s) - ‘Data/raw/Barley_MorexV3_pseudomolecules.fasta’ saved [4296032540/4296032540]



In [880]:
### --- Check the file just downloaded 
! head Data/raw/Barley_MorexV3_pseudomolecules.fasta

>chr1H
GTTCCCGCTTCGATCCAAACATTTCGAGAACCAGGGGTCCGGTTATGTGGAACTCGTCAA
AACACGCAGTTTTGGCCTATTCCGGCGAGTTTAGTAAGGTACTACTCACTGATTTTGGTT
GCCCCTATGATTCGAACGTTTTGGGAACCCCGAGGTCCGATTACGGGGAACTCGTCAAAA
CTCACAGTTTTGTCCTATTCTGGCCAGTTTTGTATGCTATTACTCACTGATTTTGGGTCC
CGCTGTGATCGAACTTTTCGGAAACCCCAGGTTTCGGGTCCGGTTACGGGGAACTCGTCA
AAACTCACAGTTTTGGTCTATTCCGGCCAGTTTTGTTCGCTATTAGTCACTGATTTTGGG
TCGTGGTGCCATCCGAACGTTTCGTGAACCCCGGGGTCGAGTTATGGGGAAATCATGAAA
ACTCGCAGTTTTGGCCTTTTCCCCCAGTTTTTTATGCTATTACTCACTGATTTTAGGTCC
CACTGCGATACAAATTTCTCAAGAACCGAGGGGTCCGGTTACCTGGAACTCATCGAAACA


In [881]:
### --- Check where the occurence of >chr_number are within the file
! grep -n '>chr' Data/raw/Barley_MorexV3_pseudomolecules.fasta

1:>chr1H
8608435:>chr2H
19701532:>chr3H
30060142:>chr4H
40232369:>chr5H
50036015:>chr6H
59399258:>chr7H
69941602:>chrUn


In [882]:
### --- Based on the previous output divide the total fasta in smaller fasta containing information about 1 chromosome each
### --- I directly run them from the terminal and not within the Notebook due to the fact within the Notebook I obtained a Broken pipe error

! head -8608434 Data/raw/Barley_MorexV3_pseudomolecules.fasta > Data/Barley_MorexV3_pseudomolecules_chr1.fasta
! tail -n +8608435 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -11093097 > Data/Barley_MorexV3_pseudomolecules_chr2.fasta
! tail -n +19701532 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -10358610 > Data/Barley_MorexV3_pseudomolecules_chr3.fasta
! tail -n +30060142 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -10172227 > Data/Barley_MorexV3_pseudomolecules_chr4.fasta
! tail -n +40232369 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -9803646 > Data/Barley_MorexV3_pseudomolecules_chr5.fasta
! tail -n +50036015 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -9363243 > Data/Barley_MorexV3_pseudomolecules_chr6.fasta
! tail -n +59399258 Data/raw/Barley_MorexV3_pseudomolecules.fasta | head -10542344 > Data/Barley_MorexV3_pseudomolecules_chr7.fasta
! tail -n +69941602 Data/raw/Barley_MorexV3_pseudomolecules.fasta > Data/Barley_MorexV3_pseudomolecules_chrUn.fasta

tail: error writing 'standard output': Broken pipe
tail: error writing 'standard output': Broken pipe
tail: error writing 'standard output': Broken pipe
tail: error writing 'standard output': Broken pipe
tail: error writing 'standard output': Broken pipe
tail: error writing 'standard output': Broken pipe


In [883]:
### --- Writing the dataframe to a tsv file
introns_UTR_Hv_MorexHC.to_csv('Output/Hv_Morex_longread/Hv_MorexHC_UTRtot.tsv', sep='\t', header = True, index=False)

In [884]:
! head Output/Hv_Morex_longread/Hv_MorexHC_UTRtot.tsv

ID	type	start	end	length
ID=HORVU.MOREX.r3.1HG0000070.1	intron five_prime_UTR	146750	146980	232
ID=HORVU.MOREX.r3.1HG0000080.1	intron five_prime_UTR	153530	153625	97
ID=HORVU.MOREX.r3.1HG0000620.1	intron five_prime_UTR	1614154	1615780	1628
ID=HORVU.MOREX.r3.1HG0000620.2	intron five_prime_UTR	1614154	1615780	1628
ID=HORVU.MOREX.r3.1HG0000620.3	intron five_prime_UTR	1614665	1615780	1117
ID=HORVU.MOREX.r3.1HG0000670.1	intron five_prime_UTR	1725555	1726142	589
ID=HORVU.MOREX.r3.1HG0000990.1	intron five_prime_UTR	2301295	2301473	180
ID=HORVU.MOREX.r3.1HG0002160.1	intron five_prime_UTR	4509034	4509755	723
ID=HORVU.MOREX.r3.1HG0002160.2	intron five_prime_UTR	4509646	4509755	111


In [885]:
### --- Creating the file with the coordinates 
! cat Output/Hv_Morex_longread/Hv_MorexHC_UTRtot.tsv | tail -n+2 | awk -F '\t' '{print $1,$3, $4}' > Output/Hv_Morex_longread/Hv_MorexHC_UTRintrons_coords.txt

In [886]:
! head Output/Hv_Morex_longread/Hv_MorexHC_UTRintrons_coords.txt

ID=HORVU.MOREX.r3.1HG0000070.1 146750 146980
ID=HORVU.MOREX.r3.1HG0000080.1 153530 153625
ID=HORVU.MOREX.r3.1HG0000620.1 1614154 1615780
ID=HORVU.MOREX.r3.1HG0000620.2 1614154 1615780
ID=HORVU.MOREX.r3.1HG0000620.3 1614665 1615780
ID=HORVU.MOREX.r3.1HG0000670.1 1725555 1726142
ID=HORVU.MOREX.r3.1HG0000990.1 2301295 2301473
ID=HORVU.MOREX.r3.1HG0002160.1 4509034 4509755
ID=HORVU.MOREX.r3.1HG0002160.2 4509646 4509755
ID=HORVU.MOREX.r3.1HG0002450.1 5067209 5067686


In [887]:
### --- Importing a library to check the time spent running processes
from timeit import default_timer as timer

In [888]:
def extract_fasta_from_coords(coordinates, chr1, chr2, chr3, chr4, chr5, chr6, chr7, chrUn, f_out):
    t_zero = timer() # Checking the computational time
    out = open(f_out, 'a')
    with open(coordinates) as coords:
        for line in coords:
            line = line.split()
            seq = ''
            if '.1H' in line[0]:     # Ceck if the coordinates refers to chr1
                first_line = int(line[1])/60 +1     # This is due to the fasta file having 60 characters/row
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)      # Decimal part * 60 to know at which character of the line start
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60    # Same as before but to know where it ends
                with open (chr1) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr1'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')
            
            elif '.2H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr2) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr2'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')
            
            elif '.3H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr3) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr3'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')

            elif '.4H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr4) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr4'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')

            elif '.5H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr5) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr5'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')

            elif '.6H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr6) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr6'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')
        
            elif '.7H' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chr7) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chr7'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')

            elif '.Un' in line[0]:
                first_line = int(line[1])/60 +1
                last_line = int(line[2])/60 +1
                interval = int(last_line) - int(first_line)
                start = int(float(str(first_line-int(first_line))[1:])*60)
                end = int(float(str(last_line - int(last_line))[1:])*60) + interval*60
                with open (chrUn) as fasta:
                    lines = fasta.readlines()
                    for i in range(int(first_line), int(last_line)+1):
                        f_line = lines[i]
                        f_line = f_line.rstrip()
                        seq += f_line
                    seq = seq[start:end+1]
                    out.write('>chrUn'+':'+line[1]+':'+line[2]+':'+line[0])
                    out.write('\n'+seq+'\n')


    out.close()
    tot = timer() - t_zero
    print('Running time (minutes): %.3f' % (tot/60))


In [889]:
f_out = 'Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta'
coords = 'Output/Hv_Morex_longread/Hv_MorexHC_UTRintrons_coords.txt'
chr1 = 'Data/Barley_MorexV3_pseudomolecules_chr1.fasta'
chr2 = 'Data/Barley_MorexV3_pseudomolecules_chr2.fasta'
chr3 = 'Data/Barley_MorexV3_pseudomolecules_chr3.fasta'
chr4 = 'Data/Barley_MorexV3_pseudomolecules_chr4.fasta'
chr5 = 'Data/Barley_MorexV3_pseudomolecules_chr5.fasta'
chr6 = 'Data/Barley_MorexV3_pseudomolecules_chr6.fasta'
chr7 = 'Data/Barley_MorexV3_pseudomolecules_chr7.fasta'
chrUn = 'Data/Barley_MorexV3_pseudomolecules_chrUn.fasta'

In [890]:
extract_fasta_from_coords(coords, chr1, chr2, chr3, chr4, chr5, chr6, chr7, chrUn, f_out)

Running time (minutes): 112.362


In [891]:
! head Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta

>chr1:146750:146980:ID=HORVU.MOREX.r3.1HG0000070.1
TACGTACTCACTCGTGTCCTCCTCGGATGCGCTACTCTGCCTCTGCTCAAGCCTAGCTCCAAATCGGAGATCAGATCGGATCGGACCAGATCTTGTTTTCTTTCTTTCTTTCTTCTTCAAGATCCCGATCCCCGCCACCCTAAACACACCACACCACACCACGCGTCGCCGACGAATTCGTCCCAAGACTTGTCCACACGGAAATCCATCGGTTTGTTCTGCTTCTTCAG
>chr1:153530:153625:ID=HORVU.MOREX.r3.1HG0000080.1
TACCTACCACTTCACTTCAGTTCAACCTGATGCTGAATGTGAATTCAGTCCACACTCACACTCACTAATTCAGTTCAAACTGATGTTTTTTGCAG
>chr1:1614154:1615780:ID=HORVU.MOREX.r3.1HG0000620.1
GTGAGCGCCGCACTCCTCCCCCGATTCGCGCCTCGTCCCTGCCTTGAAGCCAAGATCAGTTTTTTATTTACTTGCTTGGCATAAGCAACTTCCCCCTCTCCTCTCCTCTCTTGCGTGCGGCCGCGGGTTGCTTGCAGTGCGGGGGAAGGAGGATGGAGAGGAAGGAGAAGTGGTGGGATGGGGATGTGGATTTCTTTGTTTGTTTGCTTGCTTTCTGCAGTGAAGGTCGGGATTGGAATCGCGGCCGCGGCGCTGGCTTTGCCTTGCTCCTCAGCTTATGCTTCTCCCTCTTCCTTCCCTTCCGCCGGGGGAGAAGCAACCGGCACGGCTGCTCTCCCCTCCCCTGCTTAGCCGCCTATTTCAACCCAGTTCATGGCCTAGTCACCACCACCACAACACAAGCTGCTGCTGCTGCTGCTGCTCCCACATTACTCCTCCACCAACCCCCACAAACCCTAGCTTCTTCTTCCTCCTCCTCTGCTCCCTCTTCTTCTTGTTCTTGCTGCTGCTGGTGAGTC

# 5 Blasting the extracted sequences toward the 20 genome of the gene_projection directory within the pan-genome study

In [892]:
# This is needed to install local BLAST is not already installed
# Uncomment the following to make it run

# sudo apt-get install ncbi-blast+

In [893]:
### --- Downloading cds.fasta of the 20 genome of the pangenome study contained in the gene_projection directory
! wget -O Data/Akashinriki/Akashinriki.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/ebaa1e2d-e3a0-4353-a23d-399740c503cb/1/DOWNLOAD
! wget -O Data/B1K/B1K.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/e8736e9b-893f-4bd0-8e82-f46975819d97/1/DOWNLOAD
! wget -O Data/Barke_gp/Barke_gp.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/dc5b1041-4d3a-4324-9252-e6ce2aab2854/1/DOWNLOAD
! wget -O Data/Golden_Promise/Golden_Promise.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/7e11b599-d142-4eb4-bac0-20aec793a95f/1/DOWNLOAD
! wget -O Data/Hockett/Hockett.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/9f1e0fb5-ea24-431c-9888-c777c53b9e36/1/DOWNLOAD
! wget -O Data/HOR10350_gp/HOR10350_gp.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/9584516c-393d-482b-a9f8-b8c81932cf7c/1/DOWNLOAD
! wget -O Data/HOR13821/HOR13821.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/cbe49c93-fd2b-4e3a-94ff-eb65e87c9c2e/1/DOWNLOAD
! wget -O Data/HOR13942/HOR13942.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/953362d2-cb3d-4aa7-bdc6-878c356379ee/1/DOWNLOAD
! wget -O Data/HOR21599/HOR21599.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/3c8b0f43-a125-4008-8dc2-7c52bedd6bd9/1/DOWNLOAD
! wget -O Data/HOR3081/HOR3081.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/7ffd1f73-0d68-42ea-b8b4-a1cc3b265303/1/DOWNLOAD
! wget -O Data/HOR3365/HOR3365.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/49cf0f92-692c-4c72-b6d7-933a74d1b9d2/1/DOWNLOAD
! wget -O Data/HOR7552/HOR7552.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/473af275-bc69-476f-9d89-320261caa9cf/1/DOWNLOAD
! wget -O Data/HOR8148/HOR8148.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/f7358a73-1a6e-4c3b-9a25-73651ea486bc/1/DOWNLOAD
! wget -O Data/HOR9043/HOR9043.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/32caa1cd-19ee-416c-8fcc-c3a78f18f89b/1/DOWNLOAD
! wget -O Data/Igri/Igri.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/af253083-ebd3-459a-9fe8-b1257ccc7aab/1/DOWNLOAD
! wget -O Data/Morex_gp/Morex_gp.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/fad42b36-9503-462b-9b39-fe6c431c5caa/1/DOWNLOAD
! wget -O Data/OUN333/OUN333.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/6e097aae-16b4-4f9d-a02f-8f4c96b9f85e/1/DOWNLOAD
! wget -O Data/RGT_Planet/RGT_Planet.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/c7f4868c-15c7-4435-8cf0-4341b7aad14b/1/DOWNLOAD
! wget -O Data/ZDM01467/ZDM01467.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/97c4fcfc-8510-4ac1-a879-668927c0f566/1/DOWNLOAD
! wget -O Data/ZDM02064/ZDM02064.cds.fasta https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/c28021d8-dc64-4127-ab48-d259583cbbc5/1/DOWNLOAD

--2021-09-04 15:29:08--  https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/ebaa1e2d-e3a0-4353-a23d-399740c503cb/1/DOWNLOAD
Resolving doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)... 194.94.136.144
Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48684804 (46M) [text/plain]
Saving to: ‘Data/Akashinriki/Akarashinriki.cds.fasta’

Data/Akashinriki/Ak 100%[===================>]  46,43M  1,11MB/s    in 44s     

2021-09-04 15:29:53 (1,06 MB/s) - ‘Data/Akashinriki/Akarashinriki.cds.fasta’ saved [48684804/48684804]

--2021-09-04 15:29:53--  https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/e8736e9b-893f-4bd0-8e82-f46975819d97/1/DOWNLOAD
Resolving doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)... 194.94.136.144
Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200

Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49403158 (47M) [text/plain]
Saving to: ‘Data/HOR9043/HOR9043.cds.fasta’

Data/HOR9043/HOR904 100%[===================>]  47,11M  1,19MB/s    in 48s     

2021-09-04 15:38:56 (1014 KB/s) - ‘Data/HOR9043/HOR9043.cds.fasta’ saved [49403158/49403158]

--2021-09-04 15:38:56--  https://doi.ipk-gatersleben.de/DOI/c4d433dc-bf7c-4ad9-9368-69bb77837ca5/af253083-ebd3-459a-9fe8-b1257ccc7aab/1/DOWNLOAD
Resolving doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)... 194.94.136.144
Connecting to doi.ipk-gatersleben.de (doi.ipk-gatersleben.de)|194.94.136.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 49687390 (47M) [text/plain]
Saving to: ‘Data/Igri/Igri.cds.fasta’

Data/Igri/Igri.cds. 100%[===================>]  47,38M  1,47MB/s    in 31s     

2021-09-04 15:39:28 (1,52 MB/s) - ‘Data/Igri/Igri.cds.fasta’ saved [49687390/4968

In [902]:
### --- Making a database for each of the fasta previously downloaded
! makeblastdb -in Data/Akashinriki/Akashinriki.cds.fasta -dbtype nucl -parse_seqids -out Data/Akashinriki/Akashinriki_db
! makeblastdb -in Data/B1K/B1K.cds.fasta -dbtype nucl -parse_seqids -out Data/B1K/B1K_db
! makeblastdb -in Data/Barke_gp/Barke_gp.cds.fasta -dbtype nucl -parse_seqids -out Data/Barke_gp/Barke_gp_db
! makeblastdb -in Data/Golden_Promise/Golden_Promise.cds.fasta -dbtype nucl -parse_seqids -out Data/Golden_Promise/Golden_Promise_db
! makeblastdb -in Data/Hockett/Hockett.cds.fasta -dbtype nucl -parse_seqids -out Data/Hockett/Hockett_db
! makeblastdb -in Data/HOR10350_gp/HOR10350_gp.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR10350_gp/HOR10350_gp_db
! makeblastdb -in Data/HOR13821/HOR13821.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR13821/HOR13821_db
! makeblastdb -in Data/HOR13942/HOR13942.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR13942/HOR13942_db
! makeblastdb -in Data/HOR21599/HOR21599.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR21599/HOR21599_db
! makeblastdb -in Data/HOR3081/HOR3081.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR3081/HOR3081_db
! makeblastdb -in Data/HOR3365/HOR3365.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR3365/HOR3365_db
! makeblastdb -in Data/HOR7552/HOR7552.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR7552/HOR7552_db
! makeblastdb -in Data/HOR8148/HOR8148.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR8148/HOR8148_db
! makeblastdb -in Data/HOR9043/HOR9043.cds.fasta -dbtype nucl -parse_seqids -out Data/HOR9043/HOR9043_db
! makeblastdb -in Data/Igri/Igri.cds.fasta -dbtype nucl -parse_seqids -out Data/Igri/Igri_db
! makeblastdb -in Data/Morex_gp/Morex_gp.cds.fasta -dbtype nucl -parse_seqids -out Data/Morex_gp/Morex_gp_db
! makeblastdb -in Data/OUN333/OUN333.cds.fasta -dbtype nucl -parse_seqids -out Data/OUN333/OUN333_db
! makeblastdb -in Data/RGT_Planet/RGT_Planet.cds.fasta -dbtype nucl -parse_seqids -out Data/RGT_Planet/RGT_Planet_db
! makeblastdb -in Data/ZDM01467/ZDM01467.cds.fasta -dbtype nucl -parse_seqids -out Data/ZDM01467/ZDM01467_db
! makeblastdb -in Data/ZDM02064/ZDM02064.cds.fasta -dbtype nucl -parse_seqids -out Data/ZDM02064/ZDM02064_db




Building a new DB, current time: 09/04/2021 15:48:00
New DB name:   /home/barley/Test/Data/Akashinriki/Akashinriki_db
New DB title:  Data/Akashinriki/Akashinriki.cds.fasta
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 44446 sequences in 1.53034 seconds.


Building a new DB, current time: 09/04/2021 15:48:01
New DB name:   /home/barley/Test/Data/B1K/B1K_db
New DB title:  Data/B1K/B1K.cds.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/barley/Test/Data/B1K/B1K_db
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 44566 sequences in 1.37258 seconds.


Building a new DB, current time: 09/04/2021 15:48:03
New DB name:   /home/barley/Test/Data/Barke_gp/Barke_gp_db
New DB title:  Data/Barke_gp/Barke_gp.cds.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/barley/Test/Data/Barke_gp/Barke_gp_db
Keep MBits: T
Maximum file size: 100

In [904]:
### --- Blasting my query towards each of the database previously created
### --- Default outfmt values "6 qseqid sseqid pident length mismatch gapopen qstart qend sstart send evalue bitscore"
### --- https://www.metagenomics.wiki/tools/blast/blastn-output-format-6
! blastn -db Data/Akashinriki/Akashinriki_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Akashinriki/results_Akashinriki.out
! blastn -db Data/B1K/B1K_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/B1K/results_B1K.out
! blastn -db Data/Barke_gp/Barke_gp_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Barke_gp/results_Barke_gp.out
! blastn -db Data/Golden_Promise/Golden_Promise_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Golden_Promise/results_Golden_Promise.out
! blastn -db Data/Hockett/Hockett_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Hockett/results_Hockett.out
! blastn -db Data/HOR10350_gp/HOR10350_gp_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR10350_gp/results_HOR10350_gp.out
! blastn -db Data/HOR13821/HOR13821_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR13821/results_HOR13821.out
! blastn -db Data/HOR13942/HOR13942_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR13942/results_HOR13942.out
! blastn -db Data/HOR21599/HOR21599_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR21599/results_HOR21599.out
! blastn -db Data/HOR3081/HOR3081_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR3081/results_HOR3081.out
! blastn -db Data/HOR3365/HOR3365_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR3365/results_HOR3365.out
! blastn -db Data/HOR7552/HOR7552_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR7552/results_HOR7552.out
! blastn -db Data/HOR8148/HOR8148_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR8148/results_HOR8148.out
! blastn -db Data/HOR9043/HOR9043_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/HOR9043/results_HOR9043.out
! blastn -db Data/Igri/Igri_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Igri/results_Igri.out
! blastn -db Data/Morex_gp/Morex_gp_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/Morex_gp/results_Morex_gp.out
! blastn -db Data/OUN333/OUN333_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/OUN333/results_OUN333.out
! blastn -db Data/RGT_Planet/RGT_Planet_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/RGT_Planet/results_RGT_Planet.out
! blastn -db Data/ZDM01467/ZDM01467_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/ZDM01467/results_ZDM01467.out
! blastn -db Data/ZDM02064/ZDM02064_db -query Output/Hv_Morex_longread/Hv_MorexHC_UTR_introns.fasta -outfmt 6 -out Output/ZDM02064/results_ZDM02064.out

# Extract hits among the 20 genomes

In [905]:
### --- Checking how many hits are within every blastn result
! wc -l Output/*/results_*.out | sort -g

    8492 Output/Golden_Promise/results_Golden_Promise.out
    9138 Output/Akashinriki/results_Akashinriki.out
    9329 Output/HOR7552/results_HOR7552.out
    9351 Output/HOR9043/results_HOR9043.out
    9390 Output/OUN333/results_OUN333.out
    9406 Output/HOR13942/results_HOR13942.out
    9453 Output/HOR21599/results_HOR21599.out
    9465 Output/HOR8148/results_HOR8148.out
    9467 Output/ZDM01467/results_ZDM01467.out
    9515 Output/HOR3365/results_HOR3365.out
    9674 Output/HOR13821/results_HOR13821.out
    9826 Output/Morex_gp/results_Morex_gp.out
    9874 Output/RGT_Planet/results_RGT_Planet.out
    9899 Output/HOR3081/results_HOR3081.out
    9986 Output/Igri/results_Igri.out
   10313 Output/Hockett/results_Hockett.out
   10315 Output/ZDM02064/results_ZDM02064.out
   10525 Output/HOR10350_gp/results_HOR10350_gp.out
   10542 Output/B1K/results_B1K.out
   10809 Output/Barke_gp/results_Barke_gp.out
  194769 total


In [906]:
### --- This function is meant to extract the query seq-id giving hits
def extract_blast_hits(blast_out):
    hits = []
    with open(blast_out) as f:
        for line in f:
            line = line.split()
            hit = line[0].split(':')[3] # Extracting the Hv_MorexHC ID
            if hit in hits: continue
            hits.append(hit)
    return hits

In [907]:
blast_out_list = ! ls Output/*/results_*.out 
blast_out_list

['Output/Akashinriki/results_Akashinriki.out',
 'Output/B1K/results_B1K.out',
 'Output/Barke_gp/results_Barke_gp.out',
 'Output/Golden_Promise/results_Golden_Promise.out',
 'Output/Hockett/results_Hockett.out',
 'Output/HOR10350_gp/results_HOR10350_gp.out',
 'Output/HOR13821/results_HOR13821.out',
 'Output/HOR13942/results_HOR13942.out',
 'Output/HOR21599/results_HOR21599.out',
 'Output/HOR3081/results_HOR3081.out',
 'Output/HOR3365/results_HOR3365.out',
 'Output/HOR7552/results_HOR7552.out',
 'Output/HOR8148/results_HOR8148.out',
 'Output/HOR9043/results_HOR9043.out',
 'Output/Igri/results_Igri.out',
 'Output/Morex_gp/results_Morex_gp.out',
 'Output/OUN333/results_OUN333.out',
 'Output/RGT_Planet/results_RGT_Planet.out',
 'Output/ZDM01467/results_ZDM01467.out',
 'Output/ZDM02064/results_ZDM02064.out']

In [908]:
### --- This is meant to run the function extract_blast_hits on all the genomes and create 20 variables storing the list of unique IDs
### --- Furthemore I convert the list to a set
for x in (blast_out_list):
    globals()['hits_%s' % x.split('/')[1]] = extract_blast_hits(x)
    globals()['hits_%s' % x.split('/')[1]] = set(globals()['hits_%s' % x.split('/')[1]])

In [909]:
# Set does not allow duplicates, all duplicates converting a list to a set will be removed

In [910]:
### --- Using sets I can use the intersection function already implented saving only elements common to all sets taken under examination
common_hits = hits_Akashinriki.intersection(hits_B1K,hits_Barke_gp, hits_Golden_Promise,hits_Hockett, hits_HOR10350_gp, hits_HOR13821, hits_HOR13942, hits_HOR21599, hits_HOR3081, hits_HOR3365, hits_HOR7552, hits_HOR8148, hits_HOR9043, hits_Igri, hits_Morex_gp, hits_OUN333, hits_RGT_Planet, hits_ZDM01467, hits_ZDM02064)
pd.DataFrame(common_hits).rename(columns = {0: 'ID'}).to_csv('Output/common_blast_hits.txt', index = False, header = False)

In [911]:
! cat Output/common_blast_hits.txt | sort -u | wc -l

246
